<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [46]:
import mediapipe as mp
import numpy as np
import cv2
import pandas as pd
from tqdm import tqdm
import gc
from operator import add

In [47]:
mp_drawing = mp.solutions.drawing_utils
mp_holistic = mp.solutions.holistic

In [65]:
cap = cv2.VideoCapture(0)
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=1) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
            
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))
        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [66]:
results

mediapipe.python.solution_base.SolutionOutputs

In [67]:
pose = []
for res in results.pose_landmarks.landmark:
    test = np.array([res.x, res.y, res.z, res.visibility])
    pose.append(test)

In [68]:
#pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(132)
#face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(1404)
lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)

In [69]:
def extract_keypoints(results):
#    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
#    face = np.array([[res.x, res.y, res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [70]:
result_test = extract_keypoints(results)

In [71]:
result_test

array([ 7.43505716e-01,  5.80908656e-01, -2.22882434e-07,  6.98685884e-01,
        4.84532773e-01,  5.52124811e-06,  6.33862078e-01,  3.98926407e-01,
       -2.04850640e-02,  5.96809089e-01,  3.28482985e-01, -3.93200293e-02,
        5.85219800e-01,  2.58186400e-01, -5.92463352e-02,  6.23788118e-01,
        4.24972117e-01, -8.08212832e-02,  5.30919075e-01,  4.96071279e-01,
       -1.01158269e-01,  5.60930610e-01,  4.94205832e-01, -9.70083773e-02,
        5.91108024e-01,  4.75766271e-01, -9.16086733e-02,  6.39775515e-01,
        4.98824894e-01, -9.26869363e-02,  5.43323934e-01,  5.66367149e-01,
       -1.10677905e-01,  5.75817883e-01,  5.54231405e-01, -9.62681845e-02,
        6.10079765e-01,  5.35649121e-01, -8.92921165e-02,  6.54698372e-01,
        5.69881082e-01, -1.00024670e-01,  5.69783449e-01,  6.16313517e-01,
       -1.12538032e-01,  5.99672735e-01,  6.01728141e-01, -8.95816013e-02,
        6.31523788e-01,  5.83196938e-01, -7.73787871e-02,  6.67799294e-01,
        6.30341530e-01, -

In [83]:
def signvideodataframe(filepath):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(filepath)

    # Define the path to the videos directory
    path = 'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/'

    # Create a 'Path' column by concatenating the path with 'Video file' column
    df['Path'] = path + df['Video file']
    
#    df['Gloss'] = df['Gloss'].str.replace('\d+', '',regex=True)

    # Calculate the frequency of each gloss and create a 'frequency' column
    df['Frequency'] = df['Gloss'].map(df['Gloss'].value_counts())

    # Sort the DataFrame by the 'Gloss' column
    df = df.sort_values(by='Gloss')

    # Reset the index of the DataFrame
    df = df.reset_index(drop=True)

    return df

In [84]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [85]:
traindf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/train.csv')

In [86]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P31,3827306090663467-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P37,16792698524451422-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P11,6868778695018762-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P11,6870709051348651-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P50,0719792557216079-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [87]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
40149,P37,9716493262876276-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40150,P31,7550572181460327-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40151,P46,47985881750082227-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40152,P50,04671245574824856-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
40153,P51,19959052532136146-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [88]:
traindf['Path'][0]

'file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/videos/3827306090663467-1 DOLLAR.mp4'

In [89]:
cap = cv2.VideoCapture(traindf['Path'][0])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=1) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)
        # Left hand
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Right Hand
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [90]:
traindf.sample(20)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
27483,P50,46013120900663185-PUT ASIDE.mp4,PUTASIDE,G_02_007,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3082,P31,49570070972691926-BITE.mp4,BITE1,K_02_045,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,21
4426,P37,35664034264638844-BUILDING.mp4,BUILDING,A_03_088,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,14
7829,P51,7450468332790019-COUNSELOR.mp4,COUNSELOR,B_01_005,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
6028,P40,06509153784135058-CHEAT 1.mp4,CHEAT1,A_02_008,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
36149,P33,47255809726893805-TOAST.mp4,TOAST,D_02_080,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4187,P46,972950073099436-BREAKTIME.mp4,BREAKTIME,J_01_047,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,16
3785,P31,023415744741265954-BOW 2.mp4,BOW2,G_02_063,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
39611,P52,09500205774071357-seedWORM.mp4,WORM,B_03_014,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
27023,P50,5500435923648437-PRINCE.mp4,PRINCE,E_01_003,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [91]:
traindf = traindf[traindf['Frequency'] > 17]
traindf = traindf.reset_index(drop=True)

In [92]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P52,07157565148825373-seedAXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
1,P28,7179300005186042-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
2,P29,16216064841959765-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
3,P37,6193814382865199-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19
4,P40,5947453960317015-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19


In [93]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
945,P50,9161417844146778-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
946,P14,00930662603221255-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
947,P27,82063651021682-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
948,P40,5268072837528903-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20
949,P40,6363286086951516-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,20


In [94]:
traindf.describe()

,Frequency
count,950.000000
mean,19.082105
std,1.316026
min,18.000000
25%,18.000000
50%,19.000000
75%,20.000000
max,24.000000


In [95]:
#traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)

In [96]:
traindf['Gloss'].nunique()

50

In [97]:
words = traindf['Gloss'].unique()
words

array(['AXE1', 'BACKPACK1', 'BASKETBALL1', 'BEE1', 'BELT1', 'BITE1',
       'BREAKFAST1', 'CANCER1', 'CHRISTMAS1', 'CONFUSED1', 'DARK1',
       'DEAF1', 'DECIDE1', 'DEMAND1', 'DEVELOP1', 'DINNER1', 'DOG1',
       'DOWNSIZE1', 'DRAG1', 'EAT1', 'EDIT1', 'ELEVATOR1', 'FINE1',
       'FLOAT1', 'FOREIGNER1', 'GUESS1', 'HALLOWEEN1', 'HOSPITAL1',
       'HURDLE/TRIP1', 'JEWELRY', 'KNIGHT1', 'LOCK1', 'LUNCH1', 'MAPLE',
       'MEAT1', 'MECHANIC1', 'MICROSCOPE1', 'MOVIE1', 'NOON1', 'PARTY1',
       'PATIENT2', 'RAZOR2', 'RIVER1', 'ROCKINGCHAIR1', 'SHAVE1', 'SINK',
       'SQUEEZE', 'THEY1', 'TWINS1', 'WHATFOR1'], dtype=object)

In [98]:
valdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/val.csv') 

In [99]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P26,22595012150860327-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
1,P39,7421622940519235-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
2,P21,686738356933241-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
3,P12,9219095671540121-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P21,1448188216215387-5 DOLLARS.mp4,5DOLLARS,B_01_062,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [100]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
10299,P21,6959326205750493-ZOOM IN.mp4,ZOOMIN,B_02_056,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3
10300,P12,7758716133684984-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10301,P39,844134294032034-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10302,P21,5548062993721732-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
10303,P26,19366754134806952-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4


In [101]:
testdf = signvideodataframe('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/splits/test.csv')

In [102]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P18,23521769221811684-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
1,P42,023931338852502426-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
2,P49,4893817008748198-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
3,P17,13991818149960333-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
4,P49,34625615110480457-1 DOLLAR.mp4,1DOLLAR,C_02_025,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [103]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
32936,P18,4320702510886756-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32937,P9,7676354653247301-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32938,P47,5386272465310649-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32939,P18,738440364224181-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13
32940,P17,9953298353288469-ZOOM OFF.mp4,ZOOMOFF,D_01_057,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,13


In [104]:
valdf = valdf[valdf['Gloss'].isin(words)]
valdf = valdf.reset_index(drop=True)

In [105]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [106]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
186,P26,032677896012150764-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
187,P5,6523145816470133-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
188,P39,0283886564670357-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
189,P21,6142521746642153-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5
190,P12,06408604416165864-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5


In [107]:
valdf['Gloss'].nunique()

50

In [108]:
valdf.describe()

,Frequency
count,191.000000
mean,3.963351
std,0.770316
min,3.000000
25%,3.000000
50%,4.000000
75%,4.000000
max,6.000000


In [109]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P39,19778675091674147-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
1,P26,8581142177964065-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
2,P39,3877478645046861-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
3,P21,8521417940364975-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK1,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3


In [110]:
testdf = testdf[testdf['Gloss'].isin(words)]
testdf = testdf.reset_index(drop=True)

In [111]:
testdf['Gloss'].nunique()

50

In [112]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
0,P15,33721516025652254-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
1,P47,519067006979435-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
2,P35,5104381603195376-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
3,P42,44458614013793873-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15
4,P42,9604797909481075-AXE.mp4,AXE1,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,15


In [113]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency
724,P42,3448123355846451-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
725,P47,034046510887022485-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
726,P22,9359050586202402-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
727,P42,2748457214117681-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18
728,P15,540014801916062-WHAT FOR.mp4,WHATFOR1,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18


In [114]:
testdf.describe()

,Frequency
count,729.000000
mean,14.978052
std,2.229192
min,9.000000
25%,14.000000
50%,15.000000
75%,16.000000
max,20.000000


In [115]:
traindf['Gloss'] = traindf['Gloss'].str.replace('\d+', '',regex=True)
valdf['Gloss'] = valdf['Gloss'].str.replace('\d+', '',regex=True)
testdf['Gloss'] = testdf['Gloss'].str.replace('\d+', '',regex=True)

In [116]:
words = traindf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [117]:
words = valdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [118]:
words = testdf['Gloss'].unique()
words

array(['AXE', 'BACKPACK', 'BASKETBALL', 'BEE', 'BELT', 'BITE',
       'BREAKFAST', 'CANCER', 'CHRISTMAS', 'CONFUSED', 'DARK', 'DEAF',
       'DECIDE', 'DEMAND', 'DEVELOP', 'DINNER', 'DOG', 'DOWNSIZE', 'DRAG',
       'EAT', 'EDIT', 'ELEVATOR', 'FINE', 'FLOAT', 'FOREIGNER', 'GUESS',
       'HALLOWEEN', 'HOSPITAL', 'HURDLE/TRIP', 'JEWELRY', 'KNIGHT',
       'LOCK', 'LUNCH', 'MAPLE', 'MEAT', 'MECHANIC', 'MICROSCOPE',
       'MOVIE', 'NOON', 'PARTY', 'PATIENT', 'RAZOR', 'RIVER',
       'ROCKINGCHAIR', 'SHAVE', 'SINK', 'SQUEEZE', 'THEY', 'TWINS',
       'WHATFOR'], dtype=object)

In [119]:
print(traindf['Gloss'].nunique())
print(valdf['Gloss'].nunique())
print(testdf['Gloss'].nunique())

50
50
50


In [120]:
cap = cv2.VideoCapture(traindf.Path[4])
#cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(static_image_mode=False,
                          model_complexity=1) as holistic:
    while True:
        ret, frame = cap.read()
        if ret == False:
            break
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = holistic.process(frame_rgb)

        # Mano izquieda (azul)
        mp_drawing.draw_landmarks(
            frame, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        # Mano derecha (verde)
        mp_drawing.draw_landmarks(
            frame, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
            mp_drawing.DrawingSpec(color=(0, 0, 0),
                                   thickness=2,
                                   circle_radius=1),
            mp_drawing.DrawingSpec(color=(0, 92, 230), thickness=2))

        frame = cv2.flip(frame, 1)
        cv2.imshow("Frame", frame)
        if cv2.waitKey(1) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()

In [138]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=1,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [143]:
# Define a function to extract keypoints from a single video file
def extract_keypoints_from_file_flip(filepath):
    # Initialize a MediaPipe Holistic model
    holistic = mp.solutions.holistic.Holistic(static_image_mode=False,
                                              model_complexity=1,
                                              min_detection_confidence=0.5,
                                              min_tracking_confidence=0.5)

    # Open the video file
    cap = cv2.VideoCapture(filepath)

    # Initialize an empty list to store the keypoints for each frame
    keypoints = []

    # Loop through the frames of the video
    while cap.isOpened():
        # Read a frame from the video
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to RGB color space
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.flip(frame, 1)
        # Use the Holistic model to detect landmarks for the face, pose, and hands
        results = holistic.process(frame)

        # Extract the keypoints from the results object and append them to the keypoints list
        keypoints.append(extract_keypoints(results))

    # Release the video capture object and the Holistic model
    cap.release()
    holistic.close()

    return np.array(keypoints)

In [122]:
extract_keypoints_from_file(traindf.Path[0]).shape

(100, 126)

In [145]:
test=extract_keypoints_from_file_flip(traindf.Path[0])
test[30]

array([ 7.96414077e-01,  9.08715069e-01,  9.07866706e-08,  7.49651492e-01,
        8.64592910e-01, -9.06617753e-03,  7.18871355e-01,  8.13641608e-01,
       -1.42531497e-02,  7.00173020e-01,  7.75223196e-01, -2.11012661e-02,
        6.85313523e-01,  7.38832355e-01, -2.72529013e-02,  7.53686905e-01,
        7.23692119e-01,  1.10347848e-02,  7.51472771e-01,  6.68876708e-01,
        1.31220720e-03,  7.49525487e-01,  6.38742685e-01, -1.18946834e-02,
        7.46116936e-01,  6.16605282e-01, -2.20092256e-02,  7.77112484e-01,
        7.26227760e-01,  5.35163376e-03,  7.76386797e-01,  6.63936794e-01,
       -3.61846294e-03,  7.73537099e-01,  6.33621931e-01, -1.52370287e-02,
        7.68957615e-01,  6.16506398e-01, -2.34815758e-02,  7.99829900e-01,
        7.40132511e-01, -3.79953836e-03,  8.01362157e-01,  6.78541660e-01,
       -1.29196774e-02,  7.97479153e-01,  6.52502894e-01, -1.86225455e-02,
        7.91766882e-01,  6.40358806e-01, -2.22017895e-02,  8.21929336e-01,
        7.65651405e-01, -

In [147]:
test=extract_keypoints_from_file(traindf.Path[0])
test[30]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0.])

In [148]:
cap = cv2.VideoCapture(traindf.Path[0])
length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
print( length )

100


In [149]:
type(extract_keypoints_from_file(traindf.Path[643]))

numpy.ndarray

In [150]:
traindf.count()

Participant ID    950
Video file        950
Gloss             950
ASL-LEX Code      950
Path              950
Frequency         950
dtype: int64

In [151]:
def frames_from_file(filepath):
    cap = cv2.VideoCapture(filepath)
    frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    length = round(frames / fps)
    return frames, fps, length

In [152]:
frames_from_file(traindf.Path[0])

(100, 31, 3)

In [153]:
tqdm.pandas()
traindf[['Frames', 'FPS', 'Length']] = traindf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 950/950 [00:46<00:00, 20.25it/s]


In [154]:
traindf['Frames'].max()

540

In [155]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,19,89,30,3


In [156]:
traindf.describe()

,Frequency,Frames,FPS,Length
count,950.000000,950.000000,950.000000,950.000000
mean,19.082105,87.755789,29.354737,3.016842
std,1.316026,46.654745,2.197195,1.568167
min,18.000000,25.000000,11.000000,1.000000
25%,18.000000,60.000000,29.000000,2.000000
50%,19.000000,77.000000,30.000000,3.000000
75%,20.000000,97.000000,30.000000,3.000000
max,24.000000,540.000000,31.000000,18.000000


In [157]:
traindf = traindf[(traindf['Length'] > 1) & (traindf['FPS'] > 28) &(traindf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
traindf= traindf.reset_index(drop=True)
traindf.describe()

,Frequency,Frames,FPS,Length
count,804.000000,804.000000,804.000000,804.000000
mean,19.084577,76.113184,29.680348,2.593284
std,1.305343,20.091965,0.479790,0.671427
min,18.000000,45.000000,29.000000,2.000000
25%,18.000000,60.000000,29.000000,2.000000
50%,19.000000,74.000000,30.000000,2.000000
75%,20.000000,88.000000,30.000000,3.000000
max,24.000000,130.000000,31.000000,4.000000


In [158]:
traindf['Frequency'] = traindf['Gloss'].map(traindf['Gloss'].value_counts())
#traindf = traindf.sort_values(by='Frequency',)

traindf.describe()

,Frequency,Frames,FPS,Length
count,804.000000,804.000000,804.000000,804.000000
mean,16.261194,76.113184,29.680348,2.593284
std,1.722009,20.091965,0.479790,0.671427
min,13.000000,45.000000,29.000000,2.000000
25%,15.000000,60.000000,29.000000,2.000000
50%,16.000000,74.000000,30.000000,2.000000
75%,17.000000,88.000000,30.000000,3.000000
max,20.000000,130.000000,31.000000,4.000000


In [159]:
traindf.head(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,89,30,3
5,P40,929570016067665-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,112,29,4
6,P16,904767261814883-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,64,30,2
7,P43,016134052760367945-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,75,30,2
8,P40,23099527328070546-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,91,29,3
9,P37,21310388087265242-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,18,69,30,2


In [160]:
print(traindf['Gloss'].nunique())

50


In [161]:
tqdm.pandas()
valdf[['Frames', 'FPS', 'Length']] = valdf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|█████████████████████████████████████████| 191/191 [00:08<00:00, 21.42it/s]


In [162]:
valdf['Frames'].max()

207

In [163]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P39,19778675091674147-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,121,30,4
1,P26,8581142177964065-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,81,29,3
2,P39,3877478645046861-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,120,30,4
3,P21,8521417940364975-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,71,29,2
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3,93,29,3


In [164]:
valdf.describe()

,Frequency,Frames,FPS,Length
count,191.000000,191.000000,191.000000,191.000000
mean,3.963351,88.905759,29.706806,3.000000
std,0.770316,36.214006,0.456423,1.248157
min,3.000000,37.000000,29.000000,1.000000
25%,3.000000,61.500000,29.000000,2.000000
50%,4.000000,76.000000,30.000000,3.000000
75%,4.000000,117.500000,30.000000,4.000000
max,6.000000,207.000000,30.000000,7.000000


In [165]:
valdf = valdf[(valdf['Length'] > 1) & (valdf['FPS'] > 28) &(valdf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
valdf = valdf.reset_index(drop=True)
valdf.describe()

,Frequency,Frames,FPS,Length
count,162.000000,162.000000,162.000000,162.000000
mean,3.993827,78.203704,29.691358,2.623457
std,0.760001,23.787397,0.463365,0.772311
min,3.000000,45.000000,29.000000,2.000000
25%,3.000000,59.250000,29.000000,2.000000
50%,4.000000,71.500000,30.000000,2.000000
75%,4.000000,92.000000,30.000000,3.000000
max,6.000000,130.000000,30.000000,4.000000


In [166]:
valdf['Frequency'] = valdf['Gloss'].map(valdf['Gloss'].value_counts())
#valdf = valdf.sort_values(by='Frequency',)

valdf.describe()

,Frequency,Frames,FPS,Length
count,162.000000,162.000000,162.000000,162.000000
mean,3.506173,78.203704,29.691358,2.623457
std,0.907162,23.787397,0.463365,0.772311
min,2.000000,45.000000,29.000000,2.000000
25%,3.000000,59.250000,29.000000,2.000000
50%,4.000000,71.500000,30.000000,2.000000
75%,4.000000,92.000000,30.000000,3.000000
max,5.000000,130.000000,30.000000,4.000000


In [167]:
print(valdf['Gloss'].nunique())

50


In [168]:
testdf['Path'] = testdf['Path'].str.replace('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/', 'file:///home/kristian/ASL_Citizen/',regex=True)

In [169]:
tqdm.pandas()
testdf[['Frames', 'FPS', 'Length']] = testdf['Path'].progress_apply(lambda x: pd.Series(frames_from_file(x)))

100%|████████████████████████████████████████| 729/729 [00:04<00:00, 151.73it/s]


In [170]:
testdf['Frames'].max()

234

In [171]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P15,33721516025652254-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/33721...,15,196,29,7
1,P47,519067006979435-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/51906...,15,102,26,4
2,P35,5104381603195376-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/51043...,15,70,29,2
3,P42,44458614013793873-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/44458...,15,57,30,2
4,P42,9604797909481075-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/96047...,15,66,30,2


In [172]:
testdf.describe()

,Frequency,Frames,FPS,Length
count,729.000000,729.000000,729.000000,729.000000
mean,14.978052,75.192044,28.696845,2.624143
std,2.229192,27.848864,3.113689,1.003439
min,9.000000,10.000000,14.000000,0.000000
25%,14.000000,56.000000,29.000000,2.000000
50%,15.000000,73.000000,30.000000,3.000000
75%,16.000000,90.000000,30.000000,3.000000
max,20.000000,234.000000,31.000000,8.000000


In [173]:
testdf = testdf[(testdf['Length'] > 1) & (testdf['FPS'] > 28) &(testdf['Frames'] < 131) ]
#filtered_df = traindf[(traindf['Frames'] > 0) & (traindf['Frames'] < 121)]
#filtered_df = traindf[traindf['Frames'] > 0]
testdf = testdf.reset_index(drop=True)
testdf.describe()

,Frequency,Frames,FPS,Length
count,534.000000,534.000000,534.000000,534.000000
mean,15.014981,77.256554,29.702247,2.619850
std,2.217904,19.420007,0.526337,0.670792
min,9.000000,45.000000,29.000000,2.000000
25%,14.000000,61.000000,29.000000,2.000000
50%,15.000000,75.000000,30.000000,3.000000
75%,16.000000,91.000000,30.000000,3.000000
max,20.000000,130.000000,31.000000,4.000000


In [174]:
testdf['Frequency'] = testdf['Gloss'].map(testdf['Gloss'].value_counts())
#testdf = testdf.sort_values(by='Frequency',)

testdf.describe()

,Frequency,Frames,FPS,Length
count,534.000000,534.000000,534.000000,534.000000
mean,11.254682,77.256554,29.702247,2.619850
std,2.311308,19.420007,0.526337,0.670792
min,5.000000,45.000000,29.000000,2.000000
25%,10.000000,61.000000,29.000000,2.000000
50%,12.000000,75.000000,30.000000,3.000000
75%,13.000000,91.000000,30.000000,3.000000
max,15.000000,130.000000,31.000000,4.000000


In [175]:
print(testdf['Gloss'].nunique())

50


In [176]:
traindf['Path'] = traindf['Path'].str.replace('file:///media/kristian/HDD/ASL_Citizen/ASL_Citizen/', 'file:///home/kristian/ASL_Citizen/',regex=True)

In [108]:
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_keypoints.npy", keypoints_series)

100%|███████████████████████████████████████| 804/804 [1:09:21<00:00,  5.18s/it]


In [177]:
keypoints_series = traindf['Path'].progress_apply(extract_keypoints_from_file_flip)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_keypoints_flip.npy", keypoints_series)

100%|███████████████████████████████████████| 804/804 [1:10:32<00:00,  5.26s/it]


In [178]:
keypoints_series.shape

(804,)

In [110]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_keypoints.npy", keypoints_series)

In [111]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_keypoints.npy",allow_pickle=True)

In [130]:
max(keypoints.shape[0] for keypoints in keypoints_series)

130

In [129]:
min(keypoints.shape[0] for keypoints in keypoints_series)


45

In [131]:
max_len = traindf['Frames'].max()

In [132]:
max_len

130

In [116]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 126))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [115]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [116]:
keypoints_np.shape

(804, 130, 1662)

In [117]:
print(keypoints_np[6])

[[ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 [ 0.          0.          0.         ...  0.          0.
   0.        ]
 ...
 [ 0.49487716  0.39238185 -1.29218626 ...  0.          0.
   0.        ]
 [ 0.49096876  0.39365393 -1.30090356 ...  0.          0.
   0.        ]
 [ 0.48817223  0.39431143 -1.3236835  ...  0.          0.
   0.        ]]


In [119]:
keypoints_np.shape

(804, 130, 1662)

In [117]:
keypoints_series = valdf['Path'].progress_apply(extract_keypoints_from_file)

100%|█████████████████████████████████████████| 162/162 [13:46<00:00,  5.10s/it]


In [118]:
keypoints_series.shape

(162,)

In [119]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_keypoints.npy", keypoints_series)

In [120]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/val_keypoints.npy",allow_pickle=True)

In [121]:
keypoints_series.shape

(162,)

In [125]:
#keypoints_np = np.zeros((len(keypoints_series), max_len, 126))
#for i, keypoints in enumerate(keypoints_series):
#    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [126]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [127]:
keypoints_np.shape

(162, 130, 1662)

In [128]:
keypoints_np.shape

(162, 130, 1662)

In [129]:
keypoints_np

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.5595299 ,  0.3205725 , -1.27106977, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55954963,  0.32069033, -1.25959992, ...,  0.        ,
          0.        ,  0.        ],
        [ 0.55960971,  0.3209095 , -1.23169589, ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        ...,
        [ 0.5183838 ,  0.33223236, -1.15552998, ...,  

In [123]:
keypoints_series = testdf['Path'].progress_apply(extract_keypoints_from_file)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_keypoints.npy", keypoints_series)

100%|█████████████████████████████████████████| 534/534 [45:17<00:00,  5.09s/it]


In [179]:
keypoints_series = testdf['Path'].progress_apply(extract_keypoints_from_file_flip)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_keypoints_flip.npy", keypoints_series)
keypoints_series = valdf['Path'].progress_apply(extract_keypoints_from_file_flip)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_keypoints_flip.npy", keypoints_series)

100%|█████████████████████████████████████████| 162/162 [15:07<00:00,  5.60s/it]


In [113]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_keypoints.npy", keypoints_series)

(705,)

In [116]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe/test_keypoints.npy", keypoints_np)

In [127]:
import numpy as np
keypoints_series=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_keypoints.npy", allow_pickle=True)

In [128]:
keypoints_series.shape

(534,)

In [ ]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(keypoints_series), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(keypoints_series):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]

In [114]:
keypoints_np = np.zeros((len(keypoints_series), max_len, 126))
for i, keypoints in enumerate(keypoints_series):
    keypoints_np[i, :keypoints.shape[0], :] = keypoints

In [115]:
keypoints_np.shape

(705, 130, 1662)

In [118]:
keypoints_np.shape

(705, 130, 1662)

# Preprocessing

In [26]:
import numpy as np
train_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_keypoints.npy",allow_pickle=True)
val_keypoints =np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_keypoints.npy",allow_pickle=True)
test_keypoints=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_keypoints.npy",allow_pickle=True)

print('Training Keypoints Shape:', train_keypoints.shape)
print('Validation Kepoints Shape:', val_keypoints.shape)
print('Test Keypoints Shape:',test_keypoints.shape)

Training Keypoints Shape: (804,)
Validation Kepoints Shape: (162,)
Test Keypoints Shape: (534,)


In [27]:
train_keypoints_flip=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_keypoints_flip.npy",allow_pickle=True)
val_keypoints_flip =np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_keypoints_flip.npy",allow_pickle=True)
test_keypoints_flip=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_keypoints_flip.npy",allow_pickle=True)

print('Training Keypoints Shape:', train_keypoints_flip.shape)
print('Validation Kepoints Shape:', val_keypoints_flip.shape)
print('Test Keypoints Shape:',test_keypoints_flip.shape)

Training Keypoints Shape: (804,)
Validation Kepoints Shape: (162,)
Test Keypoints Shape: (534,)


In [28]:
max_len=130

In [29]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(train_keypoints), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(train_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
train_keypoints = keypoints_np
print('Training Keypoints Shape:', train_keypoints.shape)

Training Keypoints Shape: (804, 130, 126)


In [30]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(val_keypoints), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(val_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
val_keypoints = keypoints_np
print('Validation Keypoints Shape:', val_keypoints.shape)

Validation Keypoints Shape: (162, 130, 126)


In [31]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(test_keypoints), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(test_keypoints):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
test_keypoints = keypoints_np
print('Test Keypoints Shape:', test_keypoints.shape)

Test Keypoints Shape: (534, 130, 126)


In [32]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(train_keypoints_flip), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(train_keypoints_flip):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
train_keypoints_flip = keypoints_np
print('Training Keypoints Shape:', train_keypoints_flip.shape)

Training Keypoints Shape: (804, 130, 126)


In [33]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(val_keypoints_flip), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(val_keypoints_flip):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
val_keypoints_flip = keypoints_np
print('Validation Keypoints Shape:', val_keypoints_flip.shape)

Validation Keypoints Shape: (162, 130, 126)


In [34]:
# Create a padded array to store keypoints
keypoints_np = np.zeros((len(test_keypoints_flip), max_len, 126))

# Iterate through keypoints_series and fill keypoints_np with pre-padding
for i, keypoints in enumerate(test_keypoints_flip):
    keypoints_len = min(keypoints.shape[0], max_len)
    padding_len = max_len - keypoints_len
    keypoints_np[i, padding_len:, :] = keypoints[:keypoints_len, :]
    
test_keypoints_flip = keypoints_np
print('Test Keypoints Shape:', test_keypoints_flip.shape)

Test Keypoints Shape: (534, 130, 126)


In [35]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from keras.utils import to_categorical
import numpy as np
from keras.preprocessing.sequence import pad_sequences

In [140]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/07157...,18,100,31,3
1,P28,7179300005186042-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/71793...,18,81,30,3
2,P29,16216064841959765-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/16216...,18,47,30,2
3,P37,6193814382865199-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/61938...,18,67,30,2
4,P40,5947453960317015-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/59474...,18,89,30,3


In [141]:
encoder = OrdinalEncoder()

In [142]:
traindf['Cat_label'] = encoder.fit_transform(traindf[['Gloss']]).astype(int)
traindf = traindf.sort_values(by='Gloss',)

In [143]:
traindf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P52,07157565148825373-seedAXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/07157...,18,100,31,3,0
17,P37,29360158406046777-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/29360...,18,73,30,2,0
16,P27,053394218351220823-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/05339...,18,52,29,2,0
15,P40,10598328043940142-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/10598...,18,81,29,3,0
14,P50,3937261764308986-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/39372...,18,72,30,2,0


In [144]:
traindf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
789,P40,6445331634562388-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/64453...,16,89,30,3,49
788,P37,6237575353180616-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/62375...,16,56,30,2,49
802,P40,5268072837528903-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/52680...,16,92,29,3,49
794,P37,16185522171162914-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/16185...,16,64,30,2,49
803,P40,6363286086951516-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/63632...,16,65,29,2,49


In [145]:
valdf['Cat_label'] = encoder.fit_transform(valdf[['Gloss']]).astype(int)
valdf = valdf.sort_values(by='Gloss',)

In [146]:
valdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P39,19778675091674147-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,121,30,4,0
1,P26,8581142177964065-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,81,29,3,0
2,P39,3877478645046861-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,120,30,4,0
3,P21,8521417940364975-AXE.mp4,AXE,G_03_066,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,4,71,29,2,0
4,P5,9990244640190733-BACKPACK.mp4,BACKPACK,G_03_091,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,3,93,29,3,1


In [147]:
valdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
159,P39,0283886564670357-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,121,30,4,49
160,P21,6142521746642153-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,57,30,2,49
157,P26,032677896012150764-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,81,29,3,49
158,P5,6523145816470133-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,116,30,4,49
161,P12,06408604416165864-WHAT FOR.mp4,WHATFOR,C_02_054,file:///media/kristian/HDD/ASL_Citizen/ASL_Cit...,5,71,30,2,49


In [148]:
testdf['Cat_label'] = encoder.fit_transform(testdf[['Gloss']]).astype(int)
testdf = testdf.sort_values(by='Gloss',)

In [149]:
testdf.head()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
0,P35,5104381603195376-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/51043...,11,70,29,2,0
1,P42,44458614013793873-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/44458...,11,57,30,2,0
2,P42,9604797909481075-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/96047...,11,66,30,2,0
3,P9,30728048195204827-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/30728...,11,96,30,3,0
4,P6,28522130623160047-AXE.mp4,AXE,G_03_066,file:///home/kristian/ASL_Citizen/videos/28522...,11,54,29,2,0


In [150]:
testdf.tail()

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
522,P6,6734748834983595-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/67347...,13,66,29,2,49
521,P17,33051835760678294-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/33051...,13,52,31,2,49
532,P42,2748457214117681-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/27484...,13,74,30,2,49
526,P42,9684934734400592-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/96849...,13,89,30,3,49
533,P15,540014801916062-WHAT FOR.mp4,WHATFOR,C_02_054,file:///home/kristian/ASL_Citizen/videos/54001...,13,79,29,3,49


In [151]:
label_map = {label:num for num, label in enumerate(traindf['Gloss'].unique())}

In [152]:
label_map

{'AXE': 0,
 'BACKPACK': 1,
 'BASKETBALL': 2,
 'BEE': 3,
 'BELT': 4,
 'BITE': 5,
 'BREAKFAST': 6,
 'CANCER': 7,
 'CHRISTMAS': 8,
 'CONFUSED': 9,
 'DARK': 10,
 'DEAF': 11,
 'DECIDE': 12,
 'DEMAND': 13,
 'DEVELOP': 14,
 'DINNER': 15,
 'DOG': 16,
 'DOWNSIZE': 17,
 'DRAG': 18,
 'EAT': 19,
 'EDIT': 20,
 'ELEVATOR': 21,
 'FINE': 22,
 'FLOAT': 23,
 'FOREIGNER': 24,
 'GUESS': 25,
 'HALLOWEEN': 26,
 'HOSPITAL': 27,
 'HURDLE/TRIP': 28,
 'JEWELRY': 29,
 'KNIGHT': 30,
 'LOCK': 31,
 'LUNCH': 32,
 'MAPLE': 33,
 'MEAT': 34,
 'MECHANIC': 35,
 'MICROSCOPE': 36,
 'MOVIE': 37,
 'NOON': 38,
 'PARTY': 39,
 'PATIENT': 40,
 'RAZOR': 41,
 'RIVER': 42,
 'ROCKINGCHAIR': 43,
 'SHAVE': 44,
 'SINK': 45,
 'SQUEEZE': 46,
 'THEY': 47,
 'TWINS': 48,
 'WHATFOR': 49}

In [153]:
traindf.sample(10)

,Participant ID,Video file,Gloss,ASL-LEX Code,Path,Frequency,Frames,FPS,Length,Cat_label
197,P11,93340344109002-DEAF.mp4,DEAF,F_03_038,file:///home/kristian/ASL_Citizen/videos/93340...,18,52,29,2,11
190,P50,6965626273607881-DEAF.mp4,DEAF,F_03_038,file:///home/kristian/ASL_Citizen/videos/69656...,18,63,30,2,11
671,P50,37298532610460033-RAZOR 2.mp4,RAZOR,K_03_103,file:///home/kristian/ASL_Citizen/videos/37298...,13,60,30,2,41
278,P27,635034042192421-DOG.mp4,DOG,A_01_056,file:///home/kristian/ASL_Citizen/videos/63503...,20,49,30,2,16
99,P29,2296487888208607-BITE.mp4,BITE,K_02_045,file:///home/kristian/ASL_Citizen/videos/22964...,19,45,30,2,5
639,P37,9047865551380374-PARTY.mp4,PARTY,G_02_059,file:///home/kristian/ASL_Citizen/videos/90478...,20,75,30,2,39
554,P29,3817632832150144-MEAT.mp4,MEAT,D_02_022,file:///home/kristian/ASL_Citizen/videos/38176...,16,52,30,2,34
762,P27,4163876725261626-THEY.mp4,THEY,F_02_103,file:///home/kristian/ASL_Citizen/videos/41638...,14,45,30,2,47
779,P43,9855459225573504-TWINS.mp4,TWINS,F_01_032,file:///home/kristian/ASL_Citizen/videos/98554...,17,69,29,2,48
30,P40,14052296225070893-BACKPACK.mp4,BACKPACK,G_03_091,file:///home/kristian/ASL_Citizen/videos/14052...,17,88,30,3,1


In [154]:
Ytrain = encoder.fit_transform(traindf[['Gloss']])
Ytest = encoder.fit_transform(testdf[['Gloss']])
Yval = encoder.fit_transform(valdf[['Gloss']])

In [155]:
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_labels_array_1D.npy", Ytrain)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_labels_array_1D.npy", Ytest)
np.save("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_labels_array_1D.npy", Yval)

In [36]:
Ytrain=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_labels_array_1D.npy")
Ytest=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_labels_array_1D.npy")
Yval=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_labels_array_1D.npy")
Ytrain_flip=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/train_labels_array_1D.npy")
Ytest_flip=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/test_labels_array_1D.npy")
Yval_flip=np.load("/home/kristian/Capstone/keypoints/ASL_Citizen/Mediapipe_hands/val_labels_array_1D.npy")


In [37]:
Xtrain = train_keypoints
Xtest = test_keypoints
Xval = val_keypoints
Xtrain_flip = train_keypoints_flip
Xtest_flip = test_keypoints_flip
Xval_flip = val_keypoints_flip

In [88]:
Xval = val_keypoints
Xval_flip = val_keypoints_flip

In [38]:
n_classes = 50
print("Shape before one-hot encoding: ", Ytrain.shape)
Ytrain = to_categorical(Ytrain, n_classes)
print("Shape after one-hot encoding: ", Ytrain.shape)
print("Shape before one-hot encoding: ", Ytest.shape)
Ytest = to_categorical(Ytest, n_classes)
print("Shape after one-hot encoding: ", Ytest.shape)
print("Shape before one-hot encoding: ", Yval.shape)
Yval = to_categorical(Yval, n_classes)
print("Shape after one-hot encoding: ", Yval.shape)
print("Shape before one-hot encoding: ", Ytrain_flip.shape)
Ytrain_flip = to_categorical(Ytrain_flip, n_classes)
print("Shape after one-hot encoding: ", Ytrain_flip.shape)
print("Shape before one-hot encoding: ", Ytest_flip.shape)
Ytest_flip = to_categorical(Ytest_flip, n_classes)
print("Shape after one-hot encoding: ", Ytest_flip.shape)
print("Shape before one-hot encoding: ", Yval_flip.shape)
Yval_flip = to_categorical(Yval_flip, n_classes)
print("Shape after one-hot encoding: ", Yval_flip.shape)

Shape before one-hot encoding:  (804, 1)
Shape after one-hot encoding:  (804, 50)
Shape before one-hot encoding:  (534, 1)
Shape after one-hot encoding:  (534, 50)
Shape before one-hot encoding:  (162, 1)
Shape after one-hot encoding:  (162, 50)
Shape before one-hot encoding:  (804, 1)
Shape after one-hot encoding:  (804, 50)
Shape before one-hot encoding:  (534, 1)
Shape after one-hot encoding:  (534, 50)
Shape before one-hot encoding:  (162, 1)
Shape after one-hot encoding:  (162, 50)


In [39]:
type(Ytrain)

numpy.ndarray

In [40]:
Ytrain.shape

(804, 50)

In [10]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y,
                                                    test_size=0.2,
                                                    random_state=10)

In [41]:
Xtrain = np.concatenate((Xtrain, Xtest,Xtrain_flip,Xtest_flip), axis=0)
print(Xtrain.shape)
Ytrain = np.concatenate((Ytrain, Ytest, Ytrain_flip, Ytest_flip), axis=0)
print(Ytrain.shape)

(2676, 130, 126)
(2676, 50)


In [89]:
Xval.shape

(162, 130, 126)

In [90]:
Xval_flip.shape

(162, 130, 126)

In [91]:
Xval = np.concatenate((Xval, Xval_flip), axis=0)
print(Xval.shape)
Yval = np.concatenate((Yval, Yval_flip), axis=0)
print(Yval.shape)

(324, 130, 126)


In [42]:
from sklearn.preprocessing import MinMaxScaler


num_samples, num_frames, num_keypoints = Xtrain.shape
Xtrain = Xtrain.reshape(num_samples, -1)

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
Xtrain = scaler.fit_transform(Xtrain)

# Reshape the scaled data back to the original shape
Xtrain = Xtrain.reshape(num_samples, num_frames, num_keypoints)

In [92]:
from sklearn.preprocessing import MinMaxScaler


num_samples, num_frames, num_keypoints = Xval.shape
Xval = Xval.reshape(num_samples, -1)

# Create a MinMaxScaler
scaler = MinMaxScaler()

# Fit and transform the data
Xval = scaler.fit_transform(Xval)

# Reshape the scaled data back to the original shape
Xval = Xval.reshape(num_samples, num_frames, num_keypoints)

In [43]:
Xtrain.shape

(2676, 130, 126)

In [20]:
X_train, X_test, Y_train, Y_test = train_test_split(input_data_scaled,
                                                    Y,
                                                    test_size=0.,
                                                    random_state=10)

NameError: name 'input_data_scaled' is not defined

In [44]:
print(Xtrain.shape)
Xtest.shape

(2676, 130, 126)


(534, 130, 126)

In [45]:
print(Ytrain.shape)
Ytest.shape

(2676, 50)


(534, 50)

In [46]:
Xtrain

array([[[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 1.        ],
        ...,
        [0.        , 0.        , 0.65795026, ..., 0.        ,
         0.        , 0.5950842 ],
        [0.        , 0.        , 0.62974051, ..., 0.        ,
         0.        , 0.68506463],
        [0.        , 0.        , 0.5743335 , ..., 0.        ,
         0.        , 0.41467175]],

       [[0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 0.        ],
        [0.        , 0.        , 0.        , ..., 0.        ,
         0.        , 1.        ],
        ...,
        [0.        , 0.        , 0.65795026, ..., 0.        ,
         0.        , 0.5950842 ],
        [0. 

# Model

In [47]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, GRU
from tensorflow.keras.callbacks import TensorBoard
import tensorflow as tf
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam ,Adagrad, Adadelta, SGD, Lion
import os


In [48]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
tf.random.set_seed(100)

In [49]:
physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)

In [50]:
max_len = 130

In [68]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(LSTM(180, return_sequences=True, activation=act_function, input_shape=(130,126)))
model.add(LSTM(90, return_sequences=True, activation=act_function))
#model.add(LSTM(180, return_sequences=True, activation=act_function))
model.add(LSTM(90, return_sequences=False,activation=act_function))
#model.add(Dense(90, activation=act_function))
#model.add(Dense(90, activation=act_function))
#model.add(Dense(90, activation=act_function))
#model.add(Dense(90, activation=act_function))
#model.add(Dense(45, activation=act_function))
model.add(Dense(90, activation=act_function))
model.add(Dense(50, activation=act_function))
model.summary()

optimizer = SGD(learning_rate=.00001,nesterov=True)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 130, 180)          221040    
                                                                 
 lstm_3 (LSTM)               (None, 130, 90)           97560     
                                                                 
 lstm_4 (LSTM)               (None, 90)                65160     
                                                                 
 dense_10 (Dense)            (None, 90)                8190      
                                                                 
 dense_11 (Dense)            (None, 50)                4550      
                                                                 
Total params: 396500 (1.51 MB)
Trainable params: 396500 (1.51 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/1000
34/34 [

KeyboardInterrupt: 

In [170]:
model = Sequential(name='Sequential')
act_function = 'tanh'
model.add(GRU(64, return_sequences=True, activation=act_function, input_shape=(130,126)))
model.add(GRU(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation=act_function))
optimizer = Adagrad(learning_rate=.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 64)
model.summary()

Epoch 1/1000


2023-08-10 19:37:31.086458: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


17/17 [==============================] - 7s 64ms/step - loss: 8.5475 - categorical_accuracy: 0.0234 - val_loss: 8.1678 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
17/17 [==============================] - 0s 26ms/step - loss: 7.7508 - categorical_accuracy: 0.0271 - val_loss: 7.6328 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
17/17 [==============================] - 0s 26ms/step - loss: 8.9888 - categorical_accuracy: 0.0243 - val_loss: 8.2151 - val_categorical_accuracy: 0.0522
Epoch 4/1000
17/17 [==============================] - 0s 25ms/step - loss: 9.8813 - categorical_accuracy: 0.0243 - val_loss: 7.1391 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
17/17 [==============================] - 0s 25ms/step - loss: 8.5391 - categorical_accuracy: 0.0280 - val_loss: 10.0403 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
17/17 [==============================] - 0s 27ms/step - loss: 6.3260 - categorical_accuracy: 0.0290 - val_loss: 7.3463 - val_categorical_accuracy: 0

Epoch 49/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.4271 - categorical_accuracy: 0.0262 - val_loss: 6.2155 - val_categorical_accuracy: 0.0000e+00
Epoch 50/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.7637 - categorical_accuracy: 0.0252 - val_loss: 8.3997 - val_categorical_accuracy: 0.0000e+00
Epoch 51/1000
17/17 [==============================] - 0s 26ms/step - loss: 7.7397 - categorical_accuracy: 0.0224 - val_loss: 7.5216 - val_categorical_accuracy: 0.0000e+00
Epoch 52/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.5803 - categorical_accuracy: 0.0187 - val_loss: 8.2021 - val_categorical_accuracy: 0.0000e+00
Epoch 53/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.8845 - categorical_accuracy: 0.0299 - val_loss: 8.2387 - val_categorical_accuracy: 0.0000e+00
Epoch 54/1000
17/17 [==============================] - 0s 26ms/step - loss: 7.7295 - categorical_accuracy: 0.0271 - val_loss: 6.6656 - val_c

Epoch 97/1000
17/17 [==============================] - 0s 26ms/step - loss: 7.7295 - categorical_accuracy: 0.0280 - val_loss: 6.0611 - val_categorical_accuracy: 0.0000e+00
Epoch 98/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.6836 - categorical_accuracy: 0.0252 - val_loss: 5.3140 - val_categorical_accuracy: 0.0000e+00
Epoch 99/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.6254 - categorical_accuracy: 0.0271 - val_loss: 7.3098 - val_categorical_accuracy: 0.0000e+00
Epoch 100/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.7652 - categorical_accuracy: 0.0290 - val_loss: 5.8322 - val_categorical_accuracy: 0.0000e+00
Epoch 101/1000
17/17 [==============================] - 0s 25ms/step - loss: 7.6242 - categorical_accuracy: 0.0336 - val_loss: 7.9454 - val_categorical_accuracy: 0.0000e+00
Epoch 102/1000
17/17 [==============================] - 0s 26ms/step - loss: 7.5506 - categorical_accuracy: 0.0243 - val_loss: 7.8198 - va

KeyboardInterrupt: 

In [171]:
model = Sequential()

act_function = 'LeakyReLU'
model.add(LSTM(64, return_sequences=True, activation=act_function, input_shape=(130,126)))
model.add(GRU(128, return_sequences=True, activation=act_function))
model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(32, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = SGD(learning_rate=.0000005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
34/34 [==============================] - 24s 563ms/step - loss: 3.9129 - categorical_accuracy: 0.0280 - val_loss: 3.9095 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
34/34 [==============================] - 18s 534ms/step - loss: 3.9129 - categorical_accuracy: 0.0280 - val_loss: 3.9095 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
34/34 [==============================] - 18s 528ms/step - loss: 3.9129 - categorical_accuracy: 0.0280 - val_loss: 3.9095 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
34/34 [==============================] - 17s 512ms/step - loss: 3.9129 - categorical_accuracy: 0.0280 - val_loss: 3.9095 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
14/34 [===========>..................] - ETA: 10s - loss: 3.9135 - categorical_accuracy: 0.0290

KeyboardInterrupt: 

In [172]:
model = Sequential(name='Sequential') #good, overfitting

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=True, activation=act_function, input_shape=(130,126)))
model.add(GRU(128, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,126)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.4, batch_size = 64, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
13/13 [==============================] - 9s 442ms/step - loss: 3.9150 - categorical_accuracy: 0.0249 - val_loss: 3.9111 - val_categorical_accuracy: 0.0205
Epoch 2/1000
13/13 [==============================] - 5s 412ms/step - loss: 3.9105 - categorical_accuracy: 0.0237 - val_loss: 3.9111 - val_categorical_accuracy: 0.0131
Epoch 3/1000
13/13 [==============================] - 5s 410ms/step - loss: 3.9039 - categorical_accuracy: 0.0224 - val_loss: 3.9114 - val_categorical_accuracy: 0.0205
Epoch 4/1000
13/13 [==============================] - 5s 418ms/step - loss: 3.8937 - categorical_accuracy: 0.0262 - val_loss: 3.9152 - val_categorical_accuracy: 0.0205
Epoch 5/1000
13/13 [==============================] - 5s 416ms/step - loss: 3.8874 - categorical_accuracy: 0.0274 - val_loss: 3.9219 - val_categorical_accuracy: 0.0112
Epoch 6/1000
13/13 [==============================] - 5s 416ms/step - loss: 3.8808 - categorical_accuracy: 0.0187 - val_loss: 3.9215 - val_categorical_accuracy:

13/13 [==============================] - 6s 453ms/step - loss: 0.1663 - categorical_accuracy: 0.9476 - val_loss: 7.7187 - val_categorical_accuracy: 0.3582
Epoch 96/1000
13/13 [==============================] - 6s 462ms/step - loss: 0.1604 - categorical_accuracy: 0.9589 - val_loss: 7.9065 - val_categorical_accuracy: 0.3675
Epoch 97/1000
13/13 [==============================] - 6s 462ms/step - loss: 0.2348 - categorical_accuracy: 0.9377 - val_loss: 7.6548 - val_categorical_accuracy: 0.3731
Epoch 98/1000
13/13 [==============================] - 6s 461ms/step - loss: 0.2026 - categorical_accuracy: 0.9377 - val_loss: 7.6788 - val_categorical_accuracy: 0.3619
Epoch 99/1000
13/13 [==============================] - 6s 451ms/step - loss: 0.1327 - categorical_accuracy: 0.9539 - val_loss: 7.5109 - val_categorical_accuracy: 0.3638
Epoch 100/1000
13/13 [==============================] - 6s 461ms/step - loss: 0.1414 - categorical_accuracy: 0.9514 - val_loss: 7.2929 - val_categorical_accuracy: 0.3489

13/13 [==============================] - 6s 462ms/step - loss: 0.0572 - categorical_accuracy: 0.9825 - val_loss: 7.4264 - val_categorical_accuracy: 0.4347
Epoch 192/1000
13/13 [==============================] - 6s 465ms/step - loss: 0.0595 - categorical_accuracy: 0.9813 - val_loss: 8.1414 - val_categorical_accuracy: 0.4347
Epoch 193/1000
13/13 [==============================] - 6s 464ms/step - loss: 0.0871 - categorical_accuracy: 0.9713 - val_loss: 7.8970 - val_categorical_accuracy: 0.4347
Epoch 194/1000
13/13 [==============================] - 6s 472ms/step - loss: 0.0746 - categorical_accuracy: 0.9751 - val_loss: 8.1593 - val_categorical_accuracy: 0.4328
Epoch 195/1000
13/13 [==============================] - 6s 466ms/step - loss: 0.0704 - categorical_accuracy: 0.9738 - val_loss: 8.0066 - val_categorical_accuracy: 0.4515
Epoch 196/1000
13/13 [==============================] - 6s 467ms/step - loss: 0.0665 - categorical_accuracy: 0.9825 - val_loss: 8.0449 - val_categorical_accuracy: 0.

13/13 [==============================] - 6s 479ms/step - loss: 0.0561 - categorical_accuracy: 0.9863 - val_loss: 9.1304 - val_categorical_accuracy: 0.4627
Epoch 288/1000
13/13 [==============================] - 6s 472ms/step - loss: 0.0606 - categorical_accuracy: 0.9825 - val_loss: 9.1830 - val_categorical_accuracy: 0.4478
Epoch 289/1000
13/13 [==============================] - 6s 483ms/step - loss: 0.0690 - categorical_accuracy: 0.9838 - val_loss: 9.6757 - val_categorical_accuracy: 0.4664
Epoch 290/1000
13/13 [==============================] - 6s 482ms/step - loss: 0.0844 - categorical_accuracy: 0.9751 - val_loss: 9.7388 - val_categorical_accuracy: 0.4701
Epoch 291/1000
13/13 [==============================] - 6s 473ms/step - loss: 0.0720 - categorical_accuracy: 0.9763 - val_loss: 9.2409 - val_categorical_accuracy: 0.4720
Epoch 292/1000
13/13 [==============================] - 6s 474ms/step - loss: 0.0556 - categorical_accuracy: 0.9850 - val_loss: 8.9168 - val_categorical_accuracy: 0.

KeyboardInterrupt: 

In [173]:
res = model.predict(Xval)

6/6 [==============================] - 1s 66ms/step


In [174]:
np.argmax(res[100])

43

In [175]:
np.argmax(Yval[100])

31

In [52]:
model = Sequential(name='Sequential') ##best

act_function = 'LeakyReLU'
model.add(GRU(32, return_sequences=False, activation=act_function, dropout=.5,input_shape=(130,126)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function))
#model.add(Dense(64, activation=act_function))
model.add(Dense(16, activation=act_function))
model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00005,global_clipnorm=.25)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.3, batch_size = 64, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
30/30 [==============================] - 8s 215ms/step - loss: 3.9204 - categorical_accuracy: 0.0144 - val_loss: 3.9345 - val_categorical_accuracy: 0.0249
Epoch 2/1000
30/30 [==============================] - 6s 191ms/step - loss: 3.9167 - categorical_accuracy: 0.0160 - val_loss: 3.9284 - val_categorical_accuracy: 0.0386
Epoch 3/1000
30/30 [==============================] - 6s 192ms/step - loss: 3.9122 - categorical_accuracy: 0.0208 - val_loss: 3.9255 - val_categorical_accuracy: 0.0174
Epoch 4/1000
30/30 [==============================] - 6s 190ms/step - loss: 3.9099 - categorical_accuracy: 0.0203 - val_loss: 3.9239 - val_categorical_accuracy: 0.0174
Epoch 5/1000
30/30 [==============================] - 6s 193ms/step - loss: 3.9066 - categorical_accuracy: 0.0230 - val_loss: 3.9242 - val_categorical_accuracy: 0.0174
Epoch 6/1000
30/30 [==============================] - 6s 191ms/step - loss: 3.9071 - categorical_accuracy: 0.0182 - val_loss: 3.9249 - val_categorical_accuracy:

KeyboardInterrupt: 

In [51]:
model = Sequential(name='Sequential') #chingon

act_function = 'selu'
model.add(GRU(64, return_sequences=False, activation=act_function, dropout=.5,kernel_initializer='he_uniform',input_shape=(130,126)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function))
#model.add(Dense(64, activation=act_function))
model.add(Dense(16, activation=act_function))
model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.005,global_clipnorm=10)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.4, batch_size = 64, callbacks=[tb_callback])
model.summary()

2023-08-11 09:51:02.420040: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


Epoch 1/5000


2023-08-11 09:51:05.936031: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7ff908411900 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-08-11 09:51:05.936057: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX250, Compute Capability 6.1
2023-08-11 09:51:05.940241: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-08-11 09:51:05.953295: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-08-11 09:51:06.015538: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


26/26 [==============================] - 9s 241ms/step - loss: 4.0962 - categorical_accuracy: 0.0168 - val_loss: 4.0085 - val_categorical_accuracy: 0.0065
Epoch 2/5000
26/26 [==============================] - 4s 158ms/step - loss: 3.9604 - categorical_accuracy: 0.0249 - val_loss: 3.9157 - val_categorical_accuracy: 0.0336
Epoch 3/5000
26/26 [==============================] - 4s 164ms/step - loss: 3.8279 - categorical_accuracy: 0.0336 - val_loss: 3.7152 - val_categorical_accuracy: 0.0327
Epoch 4/5000
26/26 [==============================] - 4s 166ms/step - loss: 3.6075 - categorical_accuracy: 0.0561 - val_loss: 3.5930 - val_categorical_accuracy: 0.0401
Epoch 5/5000
26/26 [==============================] - 4s 171ms/step - loss: 3.4629 - categorical_accuracy: 0.0766 - val_loss: 3.6859 - val_categorical_accuracy: 0.0523
Epoch 6/5000
26/26 [==============================] - 5s 173ms/step - loss: 3.3429 - categorical_accuracy: 0.0854 - val_loss: 3.3516 - val_categorical_accuracy: 0.0682
Epoch

KeyboardInterrupt: 

In [178]:
model = Sequential(name='Sequential') 

act_function = 'LeakyReLU'
initializer = 'lecun_uniform'
model.add(GRU(32, return_sequences=True, kernel_initializer=initializer, activation=act_function, dropout=0,input_shape=(130,126)))
model.add(GRU(8, return_sequences=False, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(32, activation=act_function, kernel_initializer=initializer))
model.add(Dense(32, activation=act_function, kernel_initializer=initializer))
model.add(Dense(16, activation=act_function, kernel_initializer=initializer))
#model.add(Dropout(.3))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.005)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.3, batch_size = 64, callbacks=[tb_callback])
model.summary()

Epoch 1/1000
15/15 [==============================] - 12s 587ms/step - loss: 3.9141 - categorical_accuracy: 0.0192 - val_loss: 3.9452 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
15/15 [==============================] - 6s 426ms/step - loss: 3.8960 - categorical_accuracy: 0.0321 - val_loss: 4.0005 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
15/15 [==============================] - 6s 415ms/step - loss: 3.8835 - categorical_accuracy: 0.0278 - val_loss: 4.0428 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
15/15 [==============================] - 6s 427ms/step - loss: 3.8513 - categorical_accuracy: 0.0417 - val_loss: 3.9643 - val_categorical_accuracy: 0.0274
Epoch 5/1000
15/15 [==============================] - 6s 421ms/step - loss: 3.7220 - categorical_accuracy: 0.0524 - val_loss: 3.7845 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
15/15 [==============================] - 6s 432ms/step - loss: 3.5726 - categorical_accuracy: 0.0598 - val_loss: 3.6370 - val_cate

KeyboardInterrupt: 

In [44]:
initializer

NameError: name 'initializer' is not defined

In [45]:
model = Sequential(name='Sequential')

act_function = 'LeakyReLU'
model.add(GRU(128, return_sequences=False, activation=act_function, input_shape=(130,126)))
#model.add(GRU(128, return_sequences=True, activation=act_function))
#model.add(LSTM(64, return_sequences=False, activation=act_function))
model.add(Dense(128, activation=act_function, input_shape=(130,126)))
model.add(Dense(128, activation=act_function))
model.add(Dense(64, activation=act_function))
model.add(Dropout(.2))
model.add(Dense(50, activation='softmax'))
optimizer = Lion(learning_rate=.00025)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

history = model.fit(Xtrain, Ytrain, epochs=1000, validation_split=.2, batch_size = 32, callbacks=[tb_callback])
model.summary()

2023-08-08 10:10:09.127784: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 924736800 exceeds 10% of free system memory.
2023-08-08 10:10:09.961623: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 924736800 exceeds 10% of free system memory.


Epoch 1/1000
34/34 [==============================] - 9s 207ms/step - loss: 3.9179 - categorical_accuracy: 0.0224 - val_loss: 4.3422 - val_categorical_accuracy: 0.0000e+00
Epoch 2/1000
34/34 [==============================] - 7s 201ms/step - loss: 3.8832 - categorical_accuracy: 0.0234 - val_loss: 5.0312 - val_categorical_accuracy: 0.0000e+00
Epoch 3/1000
34/34 [==============================] - 7s 192ms/step - loss: 3.8310 - categorical_accuracy: 0.0215 - val_loss: 5.5114 - val_categorical_accuracy: 0.0000e+00
Epoch 4/1000
34/34 [==============================] - 7s 202ms/step - loss: 3.8174 - categorical_accuracy: 0.0271 - val_loss: 5.8857 - val_categorical_accuracy: 0.0000e+00
Epoch 5/1000
34/34 [==============================] - 7s 210ms/step - loss: 3.7950 - categorical_accuracy: 0.0336 - val_loss: 5.7742 - val_categorical_accuracy: 0.0000e+00
Epoch 6/1000
34/34 [==============================] - 7s 212ms/step - loss: 3.7763 - categorical_accuracy: 0.0262 - val_loss: 6.2825 - val_c

KeyboardInterrupt: 

In [43]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D

# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'selu'

# Add convolutional layers
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function, input_shape=(130, 126)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
#model.add(SpatialDropout1D(0.5))
model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(SpatialDropout1D(0.5))

#model.add(Conv1D(filters=50, kernel_size=2, activation=act_function))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(SpatialDropout1D(0.5))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
model.add(Dense(40, activation=act_function))
model.add(Dense(16, activation=act_function))
#model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dropout(0.5))
model.add(Dense(50, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.00075)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.3, batch_size = 128, callbacks=[tb_callback])

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_10 (Conv1D)          (None, 129, 50)           12650     
                                                                 
 max_pooling1d_10 (MaxPooli  (None, 64, 50)            0         
 ng1D)                                                           
                                                                 
 conv1d_11 (Conv1D)          (None, 63, 100)           10100     
                                                                 
 max_pooling1d_11 (MaxPooli  (None, 31, 100)           0         
 ng1D)                                                           
                                                                 
 conv1d_12 (Conv1D)          (None, 30, 50)            10050     
                                                                 
 max_pooling1d_12 (MaxPooli  (None, 15, 50)            0

Epoch 32/5000
15/15 [==============================] - 0s 27ms/step - loss: 2.1337 - categorical_accuracy: 0.3513 - val_loss: 2.1980 - val_categorical_accuracy: 0.3412
Epoch 33/5000
15/15 [==============================] - 0s 27ms/step - loss: 2.1407 - categorical_accuracy: 0.3679 - val_loss: 2.1737 - val_categorical_accuracy: 0.3425
Epoch 34/5000
15/15 [==============================] - 0s 27ms/step - loss: 2.0837 - categorical_accuracy: 0.3962 - val_loss: 2.1697 - val_categorical_accuracy: 0.3499
Epoch 35/5000
15/15 [==============================] - 0s 27ms/step - loss: 2.0354 - categorical_accuracy: 0.3807 - val_loss: 2.1663 - val_categorical_accuracy: 0.3412
Epoch 36/5000
15/15 [==============================] - 0s 27ms/step - loss: 2.0273 - categorical_accuracy: 0.3951 - val_loss: 2.2525 - val_categorical_accuracy: 0.3300
Epoch 37/5000
15/15 [==============================] - 0s 27ms/step - loss: 2.0277 - categorical_accuracy: 0.4026 - val_loss: 2.1512 - val_categorical_accuracy:

Epoch 81/5000
15/15 [==============================] - 0s 27ms/step - loss: 1.3167 - categorical_accuracy: 0.6044 - val_loss: 1.9463 - val_categorical_accuracy: 0.4633
Epoch 82/5000
15/15 [==============================] - 0s 27ms/step - loss: 1.3047 - categorical_accuracy: 0.6012 - val_loss: 1.7397 - val_categorical_accuracy: 0.4819
Epoch 83/5000
15/15 [==============================] - 0s 28ms/step - loss: 1.2917 - categorical_accuracy: 0.6097 - val_loss: 1.8784 - val_categorical_accuracy: 0.4732
Epoch 84/5000
15/15 [==============================] - 0s 27ms/step - loss: 1.2986 - categorical_accuracy: 0.6070 - val_loss: 1.7948 - val_categorical_accuracy: 0.4807
Epoch 85/5000
15/15 [==============================] - 0s 27ms/step - loss: 1.2531 - categorical_accuracy: 0.6295 - val_loss: 1.8634 - val_categorical_accuracy: 0.4869
Epoch 86/5000
15/15 [==============================] - 0s 27ms/step - loss: 1.2604 - categorical_accuracy: 0.6204 - val_loss: 1.7198 - val_categorical_accuracy:

15/15 [==============================] - 0s 27ms/step - loss: 0.9517 - categorical_accuracy: 0.7496 - val_loss: 1.8606 - val_categorical_accuracy: 0.5467
Epoch 130/5000
15/15 [==============================] - 0s 27ms/step - loss: 1.0017 - categorical_accuracy: 0.7197 - val_loss: 1.7908 - val_categorical_accuracy: 0.5467
Epoch 131/5000
15/15 [==============================] - 0s 29ms/step - loss: 0.9707 - categorical_accuracy: 0.7208 - val_loss: 1.9731 - val_categorical_accuracy: 0.5006
Epoch 132/5000
15/15 [==============================] - 0s 27ms/step - loss: 0.9937 - categorical_accuracy: 0.7176 - val_loss: 1.9043 - val_categorical_accuracy: 0.5342
Epoch 133/5000
15/15 [==============================] - 0s 28ms/step - loss: 0.9239 - categorical_accuracy: 0.7400 - val_loss: 1.9664 - val_categorical_accuracy: 0.4969
Epoch 134/5000
15/15 [==============================] - 0s 27ms/step - loss: 0.9445 - categorical_accuracy: 0.7309 - val_loss: 1.8663 - val_categorical_accuracy: 0.5517
E

15/15 [==============================] - 0s 27ms/step - loss: 0.8539 - categorical_accuracy: 0.7806 - val_loss: 1.7826 - val_categorical_accuracy: 0.6090
Epoch 178/5000
15/15 [==============================] - 0s 28ms/step - loss: 0.8016 - categorical_accuracy: 0.7854 - val_loss: 1.9159 - val_categorical_accuracy: 0.5766
Epoch 179/5000
15/15 [==============================] - 0s 28ms/step - loss: 0.8124 - categorical_accuracy: 0.7886 - val_loss: 1.9770 - val_categorical_accuracy: 0.5517
Epoch 180/5000
15/15 [==============================] - 0s 27ms/step - loss: 0.7867 - categorical_accuracy: 0.7912 - val_loss: 1.8922 - val_categorical_accuracy: 0.5791
Epoch 181/5000
15/15 [==============================] - 0s 29ms/step - loss: 0.7878 - categorical_accuracy: 0.7859 - val_loss: 1.8395 - val_categorical_accuracy: 0.5965
Epoch 182/5000
15/15 [==============================] - 0s 27ms/step - loss: 0.7808 - categorical_accuracy: 0.7859 - val_loss: 2.0498 - val_categorical_accuracy: 0.5629
E

...

15/15 [==============================] - 0s 28ms/step - loss: 0.7068 - categorical_accuracy: 0.8372 - val_loss: 2.0133 - val_categorical_accuracy: 0.5965
Epoch 418/5000
15/15 [==============================] - 0s 27ms/step - loss: 0.7176 - categorical_accuracy: 0.8425 - val_loss: 2.1266 - val_categorical_accuracy: 0.5928
Epoch 419/5000
15/15 [==============================] - 0s 28ms/step - loss: 0.7402 - categorical_accuracy: 0.8313 - val_loss: 2.0048 - val_categorical_accuracy: 0.6127
Epoch 420/5000
15/15 [==============================] - 0s 30ms/step - loss: 0.7084 - categorical_accuracy: 0.8340 - val_loss: 2.0383 - val_categorical_accuracy: 0.6202
Epoch 421/5000
15/15 [==============================] - 0s 28ms/step - loss: 0.7422 - categorical_accuracy: 0.8340 - val_loss: 2.0699 - val_categorical_accuracy: 0.5915
Epoch 422/5000
15/15 [==============================] - 0s 28ms/step - loss: 0.7376 - categorical_accuracy: 0.8286 - val_loss: 2.0213 - val_categorical_accuracy: 0.6065
E

15/15 [==============================] - 0s 29ms/step - loss: 1.4990 - categorical_accuracy: 0.6369 - val_loss: 2.1277 - val_categorical_accuracy: 0.5467
Epoch 466/5000
15/15 [==============================] - 0s 28ms/step - loss: 1.3938 - categorical_accuracy: 0.6514 - val_loss: 1.9833 - val_categorical_accuracy: 0.5579
Epoch 467/5000
15/15 [==============================] - 0s 29ms/step - loss: 1.3970 - categorical_accuracy: 0.6594 - val_loss: 1.9575 - val_categorical_accuracy: 0.5567
Epoch 468/5000
15/15 [==============================] - 0s 29ms/step - loss: 1.3950 - categorical_accuracy: 0.6679 - val_loss: 1.9613 - val_categorical_accuracy: 0.5554
Epoch 469/5000
15/15 [==============================] - 0s 29ms/step - loss: 1.3942 - categorical_accuracy: 0.6684 - val_loss: 1.8228 - val_categorical_accuracy: 0.5890
Epoch 470/5000
15/15 [==============================] - 0s 29ms/step - loss: 1.3293 - categorical_accuracy: 0.6765 - val_loss: 1.7895 - val_categorical_accuracy: 0.5853
E

15/15 [==============================] - 0s 29ms/step - loss: 0.8430 - categorical_accuracy: 0.8105 - val_loss: 1.9976 - val_categorical_accuracy: 0.5978
Epoch 514/5000
15/15 [==============================] - 0s 30ms/step - loss: 0.8903 - categorical_accuracy: 0.7971 - val_loss: 2.0068 - val_categorical_accuracy: 0.6077
Epoch 515/5000
15/15 [==============================] - 0s 30ms/step - loss: 0.8744 - categorical_accuracy: 0.8003 - val_loss: 1.9050 - val_categorical_accuracy: 0.5990
Epoch 516/5000
15/15 [==============================] - 0s 29ms/step - loss: 0.8878 - categorical_accuracy: 0.7960 - val_loss: 1.8941 - val_categorical_accuracy: 0.6127
Epoch 517/5000
15/15 [==============================] - 0s 29ms/step - loss: 0.8489 - categorical_accuracy: 0.8121 - val_loss: 1.9769 - val_categorical_accuracy: 0.6127
Epoch 518/5000
15/15 [==============================] - 0s 29ms/step - loss: 0.8508 - categorical_accuracy: 0.8046 - val_loss: 1.9790 - val_categorical_accuracy: 0.6052
E

15/15 [==============================] - 0s 30ms/step - loss: 0.7355 - categorical_accuracy: 0.8430 - val_loss: 2.3359 - val_categorical_accuracy: 0.5716
Epoch 562/5000
15/15 [==============================] - 0s 33ms/step - loss: 0.6797 - categorical_accuracy: 0.8468 - val_loss: 2.0596 - val_categorical_accuracy: 0.5965
Epoch 563/5000
15/15 [==============================] - 0s 30ms/step - loss: 0.7212 - categorical_accuracy: 0.8377 - val_loss: 2.2593 - val_categorical_accuracy: 0.5753
Epoch 564/5000
15/15 [==============================] - 0s 30ms/step - loss: 0.6749 - categorical_accuracy: 0.8532 - val_loss: 2.2616 - val_categorical_accuracy: 0.5853
Epoch 565/5000
15/15 [==============================] - 0s 31ms/step - loss: 0.7387 - categorical_accuracy: 0.8361 - val_loss: 2.1449 - val_categorical_accuracy: 0.6040
Epoch 566/5000
15/15 [==============================] - 0s 30ms/step - loss: 0.6872 - categorical_accuracy: 0.8521 - val_loss: 2.1336 - val_categorical_accuracy: 0.5990
E

KeyboardInterrupt: 

In [47]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D

# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'selu'

# Add convolutional layers
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function, input_shape=(130, 126)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
#model.add(Conv1D(filters=50, kernel_size=2, activation=act_function))
#model.add(MaxPooling1D(pool_size=2))
#model.add(SpatialDropout1D(0.5))
#model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
#model.add(MaxPooling1D(pool_size=2))
#model.add(SpatialDropout1D(0.5))

#model.add(Conv1D(filters=50, kernel_size=2, activation=act_function))
#model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='selu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(SpatialDropout1D(0.5))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dropout(0.5))
model.add(Dense(50, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.00075)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.3, batch_size = 128, callbacks=[tb_callback])

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_24 (Conv1D)          (None, 129, 50)           12650     
                                                                 
 max_pooling1d_24 (MaxPooli  (None, 64, 50)            0         
 ng1D)                                                           
                                                                 
 conv1d_25 (Conv1D)          (None, 62, 64)            9664      
                                                                 
 max_pooling1d_25 (MaxPooli  (None, 31, 64)            0         
 ng1D)                                                           
                                                                 
 spatial_dropout1d_7 (Spati  (None, 31, 64)            0         
 alDropout1D)                                                    
                                                        

15/15 [==============================] - 0s 22ms/step - loss: 0.5283 - categorical_accuracy: 0.8500 - val_loss: 3.0252 - val_categorical_accuracy: 0.4545
Epoch 40/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.4895 - categorical_accuracy: 0.8751 - val_loss: 3.1316 - val_categorical_accuracy: 0.4483
Epoch 41/5000
15/15 [==============================] - 0s 21ms/step - loss: 0.4893 - categorical_accuracy: 0.8585 - val_loss: 3.1520 - val_categorical_accuracy: 0.4346
Epoch 42/5000
15/15 [==============================] - 0s 21ms/step - loss: 0.4993 - categorical_accuracy: 0.8633 - val_loss: 2.9540 - val_categorical_accuracy: 0.4882
Epoch 43/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.4976 - categorical_accuracy: 0.8569 - val_loss: 3.0553 - val_categorical_accuracy: 0.4745
Epoch 44/5000
15/15 [==============================] - 0s 24ms/step - loss: 0.4879 - categorical_accuracy: 0.8644 - val_loss: 3.1656 - val_categorical_accuracy: 0.4832
Epoch 

Epoch 88/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.4221 - categorical_accuracy: 0.9092 - val_loss: 3.8767 - val_categorical_accuracy: 0.5230
Epoch 89/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.4622 - categorical_accuracy: 0.8996 - val_loss: 3.7301 - val_categorical_accuracy: 0.5280
Epoch 90/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.3792 - categorical_accuracy: 0.9151 - val_loss: 4.0435 - val_categorical_accuracy: 0.5367
Epoch 91/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.3995 - categorical_accuracy: 0.9119 - val_loss: 4.0432 - val_categorical_accuracy: 0.5392
Epoch 92/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.4471 - categorical_accuracy: 0.9055 - val_loss: 3.8701 - val_categorical_accuracy: 0.5355
Epoch 93/5000
15/15 [==============================] - 0s 22ms/step - loss: 0.4046 - categorical_accuracy: 0.9103 - val_loss: 3.7944 - val_categorical_accuracy:

KeyboardInterrupt: 

In [52]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D

# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'selu'

# Add convolutional layers
model.add(Conv1D(filters=25, kernel_size=2, activation=act_function, input_shape=(130, 126)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=25, kernel_size=3, activation='selu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(SpatialDropout1D(0.7))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
model.add(Dense(20, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
model.add(Dropout(0.2))
model.add(Dense(50, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.00075)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.3, batch_size = 128, callbacks=[tb_callback])

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_38 (Conv1D)          (None, 129, 25)           6325      
                                                                 
 max_pooling1d_38 (MaxPooli  (None, 64, 25)            0         
 ng1D)                                                           
                                                                 
 conv1d_39 (Conv1D)          (None, 63, 50)            2550      
                                                                 
 max_pooling1d_39 (MaxPooli  (None, 31, 50)            0         
 ng1D)                                                           
                                                                 
 conv1d_40 (Conv1D)          (None, 29, 25)            3775      
                                                                 
 max_pooling1d_40 (MaxPooli  (None, 14, 25)            0

Epoch 37/5000
15/15 [==============================] - 0s 19ms/step - loss: 2.5753 - categorical_accuracy: 0.2702 - val_loss: 2.5617 - val_categorical_accuracy: 0.2354
Epoch 38/5000
15/15 [==============================] - 0s 19ms/step - loss: 2.5664 - categorical_accuracy: 0.2696 - val_loss: 2.4536 - val_categorical_accuracy: 0.2615
Epoch 39/5000
15/15 [==============================] - 0s 20ms/step - loss: 2.5819 - categorical_accuracy: 0.2600 - val_loss: 2.4138 - val_categorical_accuracy: 0.2777
Epoch 40/5000
15/15 [==============================] - 0s 19ms/step - loss: 2.5370 - categorical_accuracy: 0.2712 - val_loss: 2.4642 - val_categorical_accuracy: 0.2528
Epoch 41/5000
15/15 [==============================] - 0s 19ms/step - loss: 2.5290 - categorical_accuracy: 0.2782 - val_loss: 2.3691 - val_categorical_accuracy: 0.2976
Epoch 42/5000
15/15 [==============================] - 0s 19ms/step - loss: 2.4860 - categorical_accuracy: 0.2712 - val_loss: 2.4598 - val_categorical_accuracy:

Epoch 86/5000
15/15 [==============================] - 0s 20ms/step - loss: 2.0995 - categorical_accuracy: 0.3711 - val_loss: 1.8783 - val_categorical_accuracy: 0.4296
Epoch 87/5000
15/15 [==============================] - 0s 20ms/step - loss: 2.0611 - categorical_accuracy: 0.3833 - val_loss: 1.7951 - val_categorical_accuracy: 0.4682
Epoch 88/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.9946 - categorical_accuracy: 0.4106 - val_loss: 1.7882 - val_categorical_accuracy: 0.4770
Epoch 89/5000
15/15 [==============================] - 0s 20ms/step - loss: 2.0076 - categorical_accuracy: 0.4031 - val_loss: 1.8115 - val_categorical_accuracy: 0.4707
Epoch 90/5000
15/15 [==============================] - 0s 19ms/step - loss: 2.0049 - categorical_accuracy: 0.3924 - val_loss: 1.8203 - val_categorical_accuracy: 0.4471
Epoch 91/5000
15/15 [==============================] - 0s 20ms/step - loss: 2.0117 - categorical_accuracy: 0.3924 - val_loss: 1.8210 - val_categorical_accuracy:

15/15 [==============================] - 0s 20ms/step - loss: 1.8408 - categorical_accuracy: 0.4554 - val_loss: 1.6100 - val_categorical_accuracy: 0.5019
Epoch 135/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.8683 - categorical_accuracy: 0.4261 - val_loss: 1.6312 - val_categorical_accuracy: 0.5106
Epoch 136/5000
15/15 [==============================] - 0s 22ms/step - loss: 1.7979 - categorical_accuracy: 0.4527 - val_loss: 1.6465 - val_categorical_accuracy: 0.5019
Epoch 137/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.7852 - categorical_accuracy: 0.4501 - val_loss: 1.5520 - val_categorical_accuracy: 0.5504
Epoch 138/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.8865 - categorical_accuracy: 0.4346 - val_loss: 1.5772 - val_categorical_accuracy: 0.5417
Epoch 139/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.7963 - categorical_accuracy: 0.4522 - val_loss: 1.6043 - val_categorical_accuracy: 0.5305
E

15/15 [==============================] - 0s 21ms/step - loss: 1.6682 - categorical_accuracy: 0.4864 - val_loss: 1.4969 - val_categorical_accuracy: 0.5517
Epoch 183/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.6944 - categorical_accuracy: 0.4784 - val_loss: 1.5207 - val_categorical_accuracy: 0.5405
Epoch 184/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.6382 - categorical_accuracy: 0.4891 - val_loss: 1.5049 - val_categorical_accuracy: 0.5467
Epoch 185/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.6912 - categorical_accuracy: 0.4955 - val_loss: 1.4877 - val_categorical_accuracy: 0.5579
Epoch 186/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.6767 - categorical_accuracy: 0.4917 - val_loss: 1.5129 - val_categorical_accuracy: 0.5554
Epoch 187/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.6761 - categorical_accuracy: 0.4837 - val_loss: 1.4896 - val_categorical_accuracy: 0.5255
E

...

15/15 [==============================] - 0s 21ms/step - loss: 1.0277 - categorical_accuracy: 0.6930 - val_loss: 1.7427 - val_categorical_accuracy: 0.5841
Epoch 1335/5000
15/15 [==============================] - 0s 21ms/step - loss: 1.0459 - categorical_accuracy: 0.6813 - val_loss: 1.6568 - val_categorical_accuracy: 0.6027
Epoch 1336/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.0215 - categorical_accuracy: 0.6946 - val_loss: 1.7343 - val_categorical_accuracy: 0.5704
Epoch 1337/5000
15/15 [==============================] - 0s 20ms/step - loss: 1.0455 - categorical_accuracy: 0.6898 - val_loss: 1.6967 - val_categorical_accuracy: 0.5766
Epoch 1338/5000
15/15 [==============================] - 0s 21ms/step - loss: 1.0652 - categorical_accuracy: 0.6866 - val_loss: 1.7232 - val_categorical_accuracy: 0.5890
Epoch 1339/5000
15/15 [==============================] - 0s 21ms/step - loss: 1.0314 - categorical_accuracy: 0.6893 - val_loss: 1.7443 - val_categorical_accuracy: 0.6

15/15 [==============================] - 0s 21ms/step - loss: 1.0358 - categorical_accuracy: 0.6759 - val_loss: 1.6957 - val_categorical_accuracy: 0.6002
Epoch 1383/5000
15/15 [==============================] - 0s 22ms/step - loss: 1.0346 - categorical_accuracy: 0.6829 - val_loss: 1.6571 - val_categorical_accuracy: 0.6040
Epoch 1384/5000
15/15 [==============================] - 0s 22ms/step - loss: 1.0449 - categorical_accuracy: 0.6818 - val_loss: 1.6127 - val_categorical_accuracy: 0.6027
Epoch 1385/5000
15/15 [==============================] - 0s 24ms/step - loss: 0.9780 - categorical_accuracy: 0.7064 - val_loss: 1.6898 - val_categorical_accuracy: 0.5965
Epoch 1386/5000
15/15 [==============================] - 0s 22ms/step - loss: 1.0384 - categorical_accuracy: 0.6946 - val_loss: 1.6101 - val_categorical_accuracy: 0.6164
Epoch 1387/5000
15/15 [==============================] - 0s 21ms/step - loss: 0.9954 - categorical_accuracy: 0.6935 - val_loss: 1.6720 - val_categorical_accuracy: 0.5

15/15 [==============================] - 0s 19ms/step - loss: 1.0586 - categorical_accuracy: 0.6556 - val_loss: 1.7025 - val_categorical_accuracy: 0.6015
Epoch 1431/5000
15/15 [==============================] - 0s 19ms/step - loss: 1.0143 - categorical_accuracy: 0.6951 - val_loss: 1.6708 - val_categorical_accuracy: 0.5816
Epoch 1432/5000
15/15 [==============================] - 0s 20ms/step - loss: 0.9942 - categorical_accuracy: 0.6930 - val_loss: 1.6965 - val_categorical_accuracy: 0.5965
Epoch 1433/5000
15/15 [==============================] - 0s 19ms/step - loss: 0.9989 - categorical_accuracy: 0.6930 - val_loss: 1.7009 - val_categorical_accuracy: 0.5791
Epoch 1434/5000
15/15 [==============================] - 0s 18ms/step - loss: 1.0130 - categorical_accuracy: 0.6765 - val_loss: 1.6842 - val_categorical_accuracy: 0.5978
Epoch 1435/5000
15/15 [==============================] - 0s 19ms/step - loss: 0.9938 - categorical_accuracy: 0.6957 - val_loss: 1.6977 - val_categorical_accuracy: 0.6

15/15 [==============================] - 0s 21ms/step - loss: 1.0356 - categorical_accuracy: 0.6919 - val_loss: 1.6811 - val_categorical_accuracy: 0.6214
Epoch 1479/5000
13/15 [=========================>....] - ETA: 0s - loss: 1.0040 - categorical_accuracy: 0.6935

KeyboardInterrupt: 

In [69]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D
from kerastuner import HyperModel

#supercool
# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'selu'

# Add convolutional layers
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function, input_shape=(130, 126)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=50, kernel_size=2, activation='selu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(SpatialDropout1D(0.7))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
model.add(Dropout(0.2))
model.add(Dense(50, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.0001)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.3, batch_size = 128, callbacks=[tb_callback])

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 129, 50)           12650     
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 64, 50)            0         
 g1D)                                                            
                                                                 
 conv1d_4 (Conv1D)           (None, 63, 100)           10100     
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, 31, 100)           0         
 g1D)                                                            
                                                                 
 conv1d_5 (Conv1D)           (None, 30, 50)            10050     
                                                                 
 max_pooling1d_5 (MaxPoolin  (None, 15, 50)            0

/tmp/ipykernel_96209/3719245833.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner import HyperModel


Epoch 1/5000
15/15 [==============================] - 3s 155ms/step - loss: 5.0549 - categorical_accuracy: 0.0208 - val_loss: 4.2104 - val_categorical_accuracy: 0.0162
Epoch 2/5000
15/15 [==============================] - 0s 24ms/step - loss: 4.4412 - categorical_accuracy: 0.0235 - val_loss: 4.0798 - val_categorical_accuracy: 0.0311
Epoch 3/5000
15/15 [==============================] - 0s 24ms/step - loss: 4.2043 - categorical_accuracy: 0.0235 - val_loss: 4.0172 - val_categorical_accuracy: 0.0349
Epoch 4/5000
15/15 [==============================] - 0s 23ms/step - loss: 4.1881 - categorical_accuracy: 0.0246 - val_loss: 4.0117 - val_categorical_accuracy: 0.0274
Epoch 5/5000
15/15 [==============================] - 0s 24ms/step - loss: 4.0956 - categorical_accuracy: 0.0294 - val_loss: 4.0021 - val_categorical_accuracy: 0.0299
Epoch 6/5000
15/15 [==============================] - 0s 23ms/step - loss: 4.0365 - categorical_accuracy: 0.0352 - val_loss: 3.9790 - val_categorical_accuracy: 0.03

KeyboardInterrupt: 

In [54]:
res = model.predict(Xval)

1/6 [====>.........................] - ETA: 1s

2023-08-11 10:02:04.577965: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 5.10GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


6/6 [==============================] - 1s 39ms/step


2023-08-11 10:02:04.804407: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.24GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


In [66]:
np.argmax(res[101])

18

In [65]:
np.argmax(Yval[101])

31

In [ ]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D
from kerastuner import HyperModel

#supercool hyperparameter
# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'selu'

# Add convolutional layers
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function, input_shape=(130, 126)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=100, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=50, kernel_size=2, activation='selu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(SpatialDropout1D(0.7))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
model.add(Dropout(0.2))
model.add(Dense(50, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.0001)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.3, batch_size = 128, callbacks=[tb_callback])

In [55]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D
from keras_tuner import HyperModel
from kerastuner.tuners import Hyperband

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        act_function = hp.Choice('dense_activation',values=['selu','LeakyReLU'],default='selu')
        model = Sequential()
        model.add(Conv1D(filters=hp.Choice('num_filters',values=[50, 100],default=50,),kernel_size=2,activation=act_function,input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(filters=hp.Choice('num_filters',values=[50, 100],default=50,),kernel_size=2,activation=act_function,input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(filters=hp.Choice('num_filters',values=[50, 100],default=50,),kernel_size=2,activation=act_function,input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(rate=hp.Float('dropout_1',min_value=0.0,max_value=0.7,default=0.25,step=0.05,)))
        model.add(Flatten())
        model.add(Dense(units=hp.Int('units',min_value=20,max_value=100,step=20,default=40),activation=act_function))
        model.add(Dropout(rate=hp.Float('dropout_2',min_value=0.0,max_value=0.7,default=0.25,step=0.05,)))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(Lion(hp.Float('learning_rate',min_value=1e-5,max_value=1e-3,sampling='LOG',default=1e-5)),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=(130,126), num_classes=50)

In [56]:

HYPERBAND_MAX_EPOCHS = 100
#MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_loss',
    seed=10,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='test',
    overwrite=True
)

2023-08-11 20:07:24.097328: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1739 MB memory:  -> device: 0, name: NVIDIA GeForce MX250, pci bus id: 0000:01:00.0, compute capability: 6.1


In [57]:
tuner.search_space_summary()


Search space summary
Default search space size: 6
dense_activation (Choice)
{'default': 'selu', 'conditions': [], 'values': ['selu', 'LeakyReLU'], 'ordered': False}
num_filters (Choice)
{'default': 50, 'conditions': [], 'values': [50, 100], 'ordered': True}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.7, 'step': 0.05, 'sampling': 'linear'}
units (Int)
{'default': 40, 'conditions': [], 'min_value': 20, 'max_value': 100, 'step': 20, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.7, 'step': 0.05, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [58]:
tuner.search(Xtrain, Ytrain, epochs=1000, validation_split=0.2,batch_size=96)

Trial 254 Complete [00h 07m 06s]
val_loss: 1.068617731332779

Best val_loss So Far: 0.6733157932758331
Total elapsed time: 02h 59m 04s
INFO:tensorflow:Oracle triggered exit


In [60]:
best_model = tuner.get_best_models(num_models=1)[0]

In [61]:
best_model

In [59]:
tuner.results_summary()

Results summary
Results in hyperband/test
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0247 summary
Hyperparameters:
dense_activation: LeakyReLU
num_filters: 50
dropout_1: 0.6000000000000001
units: 60
dropout_2: 0.1
learning_rate: 0.00036533709397454893
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0243
Score: 0.6733157932758331

Trial 0245 summary
Hyperparameters:
dense_activation: LeakyReLU
num_filters: 100
dropout_1: 0.65
units: 60
dropout_2: 0.25
learning_rate: 0.0006845608739459467
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0240
Score: 0.7041269540786743

Trial 0234 summary
Hyperparameters:
dense_activation: LeakyReLU
num_filters: 100
dropout_1: 0.6000000000000001
units: 80
dropout_2: 0.4
learning_rate: 0.0009243464236936878
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0230
Score: 0.7497703433036804

Trial 0252 summary

In [129]:
loss, accuracy = best_model.evaluate(Xval, Yval)

11/11 [==============================] - 0s 5ms/step - loss: 3.1132 - accuracy: 0.5216


In [63]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters)

In [95]:
import kerastuner as kt
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=30,
    hyperband_iterations=2)

In [100]:
print(tuner.get_best_hyperparameters(1)[0])

In [127]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D
from kerastuner import HyperModel

#supercool hyperparameter
# Define the CNN model
model = Sequential(name='Sequential')
act_function = 'relu'

# Add convolutional layers
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function, input_shape=(130, 126)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=50, kernel_size=2, activation=act_function))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=50, kernel_size=2, activation='selu', kernel_regularizer=regularizers.l2(l=0.001)))
model.add(MaxPooling1D(pool_size=2))

# droput layer, remove if no work
model.add(Dropout(0.6000000000000001))

# Flatten the output for the fully connected layers
model.add(Flatten())
# Add fully connected layers
model.add(Dense(60, activation=act_function))
#model.add(Dense(16, activation=act_function))
#model.add(Dense(40, activation=act_function))
#model.add(Dense(16, activation=act_function))
model.add(Dropout(0.1))
model.add(Dense(50, activation='softmax'))  # Output layer
optimizer = Lion(learning_rate=.00036533709397454893)
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['categorical_accuracy'])

# Print model summary
model.summary()

# Train the model
history = model.fit(Xtrain, Ytrain, epochs=5000, validation_split=.3, batch_size = 96, callbacks=[tb_callback])

Model: "Sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_15 (Conv1D)          (None, 129, 50)           12650     
                                                                 
 max_pooling1d_15 (MaxPooli  (None, 64, 50)            0         
 ng1D)                                                           
                                                                 
 conv1d_16 (Conv1D)          (None, 63, 50)            5050      
                                                                 
 max_pooling1d_16 (MaxPooli  (None, 31, 50)            0         
 ng1D)                                                           
                                                                 
 conv1d_17 (Conv1D)          (None, 30, 50)            5050      
                                                                 
 max_pooling1d_17 (MaxPooli  (None, 15, 50)            0

20/20 [==============================] - 0s 18ms/step - loss: 1.4310 - categorical_accuracy: 0.5670 - val_loss: 1.9061 - val_categorical_accuracy: 0.4608
Epoch 38/5000
20/20 [==============================] - 0s 17ms/step - loss: 1.3837 - categorical_accuracy: 0.5990 - val_loss: 1.8142 - val_categorical_accuracy: 0.4707
Epoch 39/5000
20/20 [==============================] - 0s 17ms/step - loss: 1.3540 - categorical_accuracy: 0.5782 - val_loss: 1.8330 - val_categorical_accuracy: 0.4521
Epoch 40/5000
20/20 [==============================] - 0s 17ms/step - loss: 1.3199 - categorical_accuracy: 0.5830 - val_loss: 1.7698 - val_categorical_accuracy: 0.4732
Epoch 41/5000
20/20 [==============================] - 0s 17ms/step - loss: 1.3350 - categorical_accuracy: 0.5873 - val_loss: 1.8028 - val_categorical_accuracy: 0.4832
Epoch 42/5000
20/20 [==============================] - 0s 18ms/step - loss: 1.2654 - categorical_accuracy: 0.5905 - val_loss: 1.7926 - val_categorical_accuracy: 0.4844
Epoch 

Epoch 86/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.6928 - categorical_accuracy: 0.7784 - val_loss: 1.3098 - val_categorical_accuracy: 0.6177
Epoch 87/5000
20/20 [==============================] - 0s 20ms/step - loss: 0.6873 - categorical_accuracy: 0.7886 - val_loss: 1.2555 - val_categorical_accuracy: 0.6451
Epoch 88/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.6797 - categorical_accuracy: 0.7902 - val_loss: 1.2434 - val_categorical_accuracy: 0.6501
Epoch 89/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.7178 - categorical_accuracy: 0.7758 - val_loss: 1.3652 - val_categorical_accuracy: 0.6276
Epoch 90/5000
20/20 [==============================] - 0s 17ms/step - loss: 0.6855 - categorical_accuracy: 0.7891 - val_loss: 1.4229 - val_categorical_accuracy: 0.6102
Epoch 91/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.6764 - categorical_accuracy: 0.7832 - val_loss: 1.2223 - val_categorical_accuracy:

20/20 [==============================] - 0s 18ms/step - loss: 0.5502 - categorical_accuracy: 0.8302 - val_loss: 1.0918 - val_categorical_accuracy: 0.7298
Epoch 135/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.5192 - categorical_accuracy: 0.8409 - val_loss: 1.0639 - val_categorical_accuracy: 0.7397
Epoch 136/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.4927 - categorical_accuracy: 0.8516 - val_loss: 1.0688 - val_categorical_accuracy: 0.7173
Epoch 137/5000
20/20 [==============================] - 0s 17ms/step - loss: 0.4689 - categorical_accuracy: 0.8526 - val_loss: 1.0887 - val_categorical_accuracy: 0.7298
Epoch 138/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.5309 - categorical_accuracy: 0.8553 - val_loss: 1.1013 - val_categorical_accuracy: 0.7248
Epoch 139/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.4976 - categorical_accuracy: 0.8569 - val_loss: 1.1125 - val_categorical_accuracy: 0.7210
E

20/20 [==============================] - 0s 18ms/step - loss: 0.4378 - categorical_accuracy: 0.8649 - val_loss: 1.0100 - val_categorical_accuracy: 0.7497
Epoch 183/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.4317 - categorical_accuracy: 0.8788 - val_loss: 1.0291 - val_categorical_accuracy: 0.7671
Epoch 184/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.4091 - categorical_accuracy: 0.8761 - val_loss: 1.0497 - val_categorical_accuracy: 0.7534
Epoch 185/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.4031 - categorical_accuracy: 0.8788 - val_loss: 1.0914 - val_categorical_accuracy: 0.7422
Epoch 186/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.4085 - categorical_accuracy: 0.8799 - val_loss: 1.1177 - val_categorical_accuracy: 0.7634
Epoch 187/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3776 - categorical_accuracy: 0.8809 - val_loss: 1.0428 - val_categorical_accuracy: 0.7684
E

20/20 [==============================] - 0s 18ms/step - loss: 0.3616 - categorical_accuracy: 0.8938 - val_loss: 1.0939 - val_categorical_accuracy: 0.7821
Epoch 231/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3223 - categorical_accuracy: 0.9082 - val_loss: 1.1489 - val_categorical_accuracy: 0.7858
Epoch 232/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.3453 - categorical_accuracy: 0.9002 - val_loss: 1.1978 - val_categorical_accuracy: 0.7671
Epoch 233/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3505 - categorical_accuracy: 0.8954 - val_loss: 1.0368 - val_categorical_accuracy: 0.8070
Epoch 234/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3553 - categorical_accuracy: 0.8980 - val_loss: 1.2456 - val_categorical_accuracy: 0.7522
Epoch 235/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.3614 - categorical_accuracy: 0.8884 - val_loss: 1.2046 - val_categorical_accuracy: 0.7435
E

20/20 [==============================] - 0s 19ms/step - loss: 0.3261 - categorical_accuracy: 0.9092 - val_loss: 1.3620 - val_categorical_accuracy: 0.7870
Epoch 279/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3199 - categorical_accuracy: 0.9092 - val_loss: 1.3133 - val_categorical_accuracy: 0.7783
Epoch 280/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3432 - categorical_accuracy: 0.8932 - val_loss: 1.3060 - val_categorical_accuracy: 0.7758
Epoch 281/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.3366 - categorical_accuracy: 0.9076 - val_loss: 1.2828 - val_categorical_accuracy: 0.7920
Epoch 282/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.3437 - categorical_accuracy: 0.9044 - val_loss: 1.3679 - val_categorical_accuracy: 0.7584
Epoch 283/5000
20/20 [==============================] - 0s 18ms/step - loss: 0.3166 - categorical_accuracy: 0.9039 - val_loss: 1.4399 - val_categorical_accuracy: 0.7260
E

20/20 [==============================] - 0s 19ms/step - loss: 0.3130 - categorical_accuracy: 0.9092 - val_loss: 1.5076 - val_categorical_accuracy: 0.7833
Epoch 327/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.2840 - categorical_accuracy: 0.9156 - val_loss: 1.5052 - val_categorical_accuracy: 0.7945
Epoch 328/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.2885 - categorical_accuracy: 0.9188 - val_loss: 1.5589 - val_categorical_accuracy: 0.7733
Epoch 329/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.3546 - categorical_accuracy: 0.9012 - val_loss: 1.5317 - val_categorical_accuracy: 0.7621
Epoch 330/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.3139 - categorical_accuracy: 0.9060 - val_loss: 1.4925 - val_categorical_accuracy: 0.7709
Epoch 331/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.2748 - categorical_accuracy: 0.9194 - val_loss: 1.5755 - val_categorical_accuracy: 0.7721
E

20/20 [==============================] - 0s 19ms/step - loss: 0.3232 - categorical_accuracy: 0.9130 - val_loss: 1.7039 - val_categorical_accuracy: 0.7895
Epoch 375/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.2945 - categorical_accuracy: 0.9076 - val_loss: 1.5691 - val_categorical_accuracy: 0.8007
Epoch 376/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.2922 - categorical_accuracy: 0.9194 - val_loss: 1.6485 - val_categorical_accuracy: 0.7908
Epoch 377/5000
20/20 [==============================] - 0s 19ms/step - loss: 0.2983 - categorical_accuracy: 0.9215 - val_loss: 1.6856 - val_categorical_accuracy: 0.8020
Epoch 378/5000
20/20 [==============================] - 0s 20ms/step - loss: 0.3379 - categorical_accuracy: 0.9060 - val_loss: 1.6834 - val_categorical_accuracy: 0.7958
Epoch 379/5000
20/20 [==============================] - 0s 20ms/step - loss: 0.3425 - categorical_accuracy: 0.9098 - val_loss: 1.6239 - val_categorical_accuracy: 0.8070
E

KeyboardInterrupt: 

In [110]:
res = model.predict(Xval)

11/11 [==============================] - 0s 4ms/step


In [137]:
np.argmax(res[100])

21

In [138]:
np.argmax(Yval[100])

31

In [130]:
res = best_model.predict(Xval)

11/11 [==============================] - 0s 5ms/step


In [141]:
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, SpatialDropout1D
from keras_tuner import HyperModel
from kerastuner.tuners import Hyperband

class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        act_function = hp.Choice('dense_activation',values=['relu','selu','LeakyReLU'],default='selu')
        model = Sequential()
        model.add(Conv1D(filters=hp.Choice('num_filters_1',values=[25,50, 100],default=50,),kernel_size=2,activation=act_function,input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(filters=hp.Choice('num_filters_2',values=[25,50, 100],default=50,),kernel_size=2,activation=act_function,input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Conv1D(filters=hp.Choice('num_filters_3',values=[25,50, 100],default=50,),kernel_size=2,activation=act_function,input_shape=self.input_shape))
        model.add(MaxPooling1D(pool_size=2))
        model.add(Dropout(rate=hp.Float('dropout_1',min_value=0.0,max_value=0.7,default=0.25,step=0.05,)))
        model.add(Flatten())
        model.add(Dense(units=hp.Int('units',min_value=20,max_value=100,step=20,default=40),activation=act_function))
        model.add(Dropout(rate=hp.Float('dropout_2',min_value=0.0,max_value=0.7,default=0.25,step=0.05,)))
        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(Lion(hp.Float('learning_rate',min_value=1e-5,max_value=1e-3,sampling='LOG',default=1e-5)),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        return model

hypermodel = CNNHyperModel(input_shape=(130,126), num_classes=50)

In [142]:

HYPERBAND_MAX_EPOCHS = 100
#MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2

tuner = Hyperband(
    hypermodel,
    max_epochs=HYPERBAND_MAX_EPOCHS,
    objective='val_loss',
    seed=10,
    executions_per_trial=EXECUTION_PER_TRIAL,
    directory='hyperband',
    project_name='test2',
    overwrite=True
)

In [143]:
tuner.search_space_summary()


Search space summary
Default search space size: 8
dense_activation (Choice)
{'default': 'selu', 'conditions': [], 'values': ['relu', 'selu', 'LeakyReLU'], 'ordered': False}
num_filters_1 (Choice)
{'default': 50, 'conditions': [], 'values': [25, 50, 100], 'ordered': True}
num_filters_2 (Choice)
{'default': 50, 'conditions': [], 'values': [25, 50, 100], 'ordered': True}
num_filters_3 (Choice)
{'default': 50, 'conditions': [], 'values': [25, 50, 100], 'ordered': True}
dropout_1 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.7, 'step': 0.05, 'sampling': 'linear'}
units (Int)
{'default': 40, 'conditions': [], 'min_value': 20, 'max_value': 100, 'step': 20, 'sampling': 'linear'}
dropout_2 (Float)
{'default': 0.25, 'conditions': [], 'min_value': 0.0, 'max_value': 0.7, 'step': 0.05, 'sampling': 'linear'}
learning_rate (Float)
{'default': 1e-05, 'conditions': [], 'min_value': 1e-05, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [ ]:
tuner.search(Xtrain, Ytrain, epochs=1000, validation_split=0.15,batch_size=96)

Trial 10 Complete [00h 00m 14s]
val_loss: 3.913244128227234

Best val_loss So Far: 3.9080971479415894
Total elapsed time: 00h 02m 26s

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
relu              |relu              |dense_activation
25                |100               |num_filters_1
50                |100               |num_filters_2
100               |25                |num_filters_3
0.35              |0.65              |dropout_1
80                |100               |units
0.25              |0.45              |dropout_2
2.4495e-05        |5.6745e-05        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
24/24 [==============================] - 4s 118ms/step - loss: 3.9488 - accuracy: 0.0198 - val_loss: 3.9209 - val_accuracy: 0.0174
Epoch 2/2
21/24 [============

In [60]:
best_model = tuner.get_best_models(num_models=1)[0]

In [61]:
best_model

In [59]:
tuner.results_summary()

Results summary
Results in hyperband/test
Showing 10 best trials
Objective(name="val_loss", direction="min")

Trial 0247 summary
Hyperparameters:
dense_activation: LeakyReLU
num_filters: 50
dropout_1: 0.6000000000000001
units: 60
dropout_2: 0.1
learning_rate: 0.00036533709397454893
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0243
Score: 0.6733157932758331

Trial 0245 summary
Hyperparameters:
dense_activation: LeakyReLU
num_filters: 100
dropout_1: 0.65
units: 60
dropout_2: 0.25
learning_rate: 0.0006845608739459467
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 1
tuner/round: 1
tuner/trial_id: 0240
Score: 0.7041269540786743

Trial 0234 summary
Hyperparameters:
dense_activation: LeakyReLU
num_filters: 100
dropout_1: 0.6000000000000001
units: 80
dropout_2: 0.4
learning_rate: 0.0009243464236936878
tuner/epochs: 100
tuner/initial_epoch: 34
tuner/bracket: 2
tuner/round: 2
tuner/trial_id: 0230
Score: 0.7497703433036804

Trial 0252 summary